# Medical Diagnosis Machine Learning Model

This notebook builds a simple machine learning model to predict medical diagnoses using the medical training and test datasets.

## Steps:
1. Load and explore data
2. Basic data preprocessing 
3. Split features and target
4. Train a simple model
5. Test the model
6. Evaluate results

## Step 1: Import Libraries and Load Data

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import pickle

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.utils import check_random_state

# Try importing optional libraries with availability checks
try:
    from imblearn.combine import SMOTETomek
    from imblearn.over_sampling import SMOTE, RandomOverSampler
    from imblearn.under_sampling import TomekLinks
    IMBLEARN_AVAILABLE = True
    print("✅ imbalanced-learn is available")
except ImportError:
    IMBLEARN_AVAILABLE = False
    print("⚠️ imbalanced-learn not available - install with: pip install imbalanced-learn")

try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
    print("✅ XGBoost is available")
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠️ XGBoost not available - install with: pip install xgboost")

try:
    from scipy.stats import randint, uniform
    SCIPY_AVAILABLE = True
    print("✅ SciPy is available")
except ImportError:
    SCIPY_AVAILABLE = False
    print("⚠️ SciPy not available - install with: pip install scipy")

# Additional utility imports
from collections import Counter

# Configure warnings and display options
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.neighbors')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')

# Load the datasets
print("Loading datasets...")
train_data = pd.read_csv('medical_train_dataset.csv')
test_data = pd.read_csv('medical_test_dataset.csv')

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print("\nTraining data columns:")
print(train_data.columns.tolist())

## Step 2: Comprehensive Exploratory Data Analysis (EDA)

We'll perform a thorough analysis of our medical dataset including:
1. Data overview and basic information
2. Missing value detection and visualization
3. Numerical feature distributions and outlier detection
4. Categorical feature analysis
5. Target variable distribution analysis
6. Correlation analysis between numerical features

### 2.1 Data Overview and Basic Information

Let's start by examining the basic structure of our dataset including:
- Dataset dimensions (rows × columns)
- Memory usage and data types
- First few rows to understand the data structure
- Statistical summary of numerical features
- Column categorization by data type

This foundational analysis helps us understand what we're working with before diving deeper.

In [ ]:
# 1. DATA OVERVIEW AND INFO
print("="*60)
print("🔍 1. DATA OVERVIEW AND BASIC INFORMATION")
print("="*60)

print("📊 Training Dataset Overview:")
print(f"Shape: {train_data.shape[0]} rows × {train_data.shape[1]} columns")
print(f"Memory usage: {train_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n📋 Column Information:")
print(train_data.info())

print("\n📝 First 5 rows of training data:")
print(train_data.head())

print("\n📈 Basic Statistical Summary:")
print(train_data.describe())

print("\n🏷️ Data Types Summary:")
for dtype in train_data.dtypes.value_counts().index:
    cols = train_data.select_dtypes(include=[dtype]).columns.tolist()
    print(f"{dtype}: {len(cols)} columns - {cols}")

### 2.2 Missing Values Detection and Visualization

Missing data can significantly impact our model's performance. In this section we will:
- Calculate missing value counts and percentages for each column
- Create visualizations to identify missing data patterns
- Generate a heatmap to visualize missing value distribution across the dataset
- Assess the overall data completeness

Understanding missing data patterns helps us choose appropriate imputation strategies later in preprocessing.

In [ ]:
# 2. MISSING VALUES DETECTION AND VISUALIZATION
print("="*60)
print("🕳️ 2. MISSING VALUES ANALYSIS")
print("="*60)

# Calculate missing values
missing_data = train_data.isnull().sum()
missing_percent = (missing_data / len(train_data)) * 100

# Create missing values summary
missing_summary = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percent.values
}).sort_values('Missing_Count', ascending=False)

print("📊 Missing Values Summary:")
print(missing_summary[missing_summary['Missing_Count'] > 0])

# Visualize missing values
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Missing values count plot
missing_cols = missing_summary[missing_summary['Missing_Count'] > 0]
if len(missing_cols) > 0:
    axes[0].bar(missing_cols['Column'], missing_cols['Missing_Count'])
    axes[0].set_title('Missing Values Count by Column')
    axes[0].set_xlabel('Columns')
    axes[0].set_ylabel('Missing Count')
    axes[0].tick_params(axis='x', rotation=45)
else:
    axes[0].text(0.5, 0.5, 'No Missing Values Found!', 
                ha='center', va='center', transform=axes[0].transAxes, fontsize=14)
    axes[0].set_title('Missing Values Count')

# Missing values heatmap
sns.heatmap(train_data.isnull(), yticklabels=False, cbar=True, cmap='viridis', ax=axes[1])
axes[1].set_title('Missing Values Heatmap')

plt.tight_layout()
plt.show()

print(f"\n✅ Total missing values: {missing_data.sum()}")
print(f"📊 Percentage of complete records: {(1 - train_data.isnull().any(axis=1).sum()/len(train_data))*100:.2f}%")

### 2.3 Numerical Features Distribution Analysis

Numerical features often contain the most predictive power in medical datasets. We'll analyze:
- **Distribution Patterns**: Histograms with mean/median markers to understand data spread
- **Statistical Properties**: Mean, median, standard deviation, and skewness for each feature
- **Outlier Detection**: Box plots with IQR-based outlier identification
- **Data Quality**: Identify potential data entry errors or unusual patterns

This analysis helps us understand feature characteristics and decide on appropriate scaling and transformation methods.

In [ ]:
# 3. NUMERICAL FEATURES ANALYSIS
print("="*60)
print("📊 3. NUMERICAL FEATURES DISTRIBUTION ANALYSIS")
print("="*60)

# Identify numerical columns
numerical_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
print(f"📈 Numerical columns ({len(numerical_cols)}): {numerical_cols}")

# Create subplots for histograms
n_cols = 3
n_rows = (len(numerical_cols) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes

print("\n📊 Distribution Statistics:")
for i, col in enumerate(numerical_cols):
    if i < len(axes):
        # Histogram with KDE
        axes[i].hist(train_data[col].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        axes[i].set_title(f'Distribution of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')
        
        # Add mean and median lines
        mean_val = train_data[col].mean()
        median_val = train_data[col].median()
        axes[i].axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')
        axes[i].axvline(median_val, color='green', linestyle='--', label=f'Median: {median_val:.2f}')
        axes[i].legend()
        
        # Print statistics
        print(f"  {col}:")
        print(f"    Mean: {mean_val:.2f}, Median: {median_val:.2f}")
        print(f"    Std: {train_data[col].std():.2f}, Skewness: {train_data[col].skew():.2f}")

# Remove empty subplots
for i in range(len(numerical_cols), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# Box plots for outlier detection
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes

print("\n🎯 Outlier Detection (using IQR method):")
for i, col in enumerate(numerical_cols):
    if i < len(axes):
        # Box plot
        box_plot = axes[i].boxplot(train_data[col].dropna(), patch_artist=True)
        box_plot['boxes'][0].set_facecolor('lightblue')
        axes[i].set_title(f'Box Plot: {col}')
        axes[i].set_ylabel(col)
        
        # Calculate outliers using IQR method
        Q1 = train_data[col].quantile(0.25)
        Q3 = train_data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = train_data[(train_data[col] < lower_bound) | (train_data[col] > upper_bound)][col]
        print(f"  {col}: {len(outliers)} outliers ({len(outliers)/len(train_data)*100:.1f}%)")

# Remove empty subplots
for i in range(len(numerical_cols), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### 2.4 Categorical Features Analysis

Categorical variables provide important contextual information in medical diagnosis. We'll examine:
- **Unique Value Counts**: Number of distinct categories per feature
- **Frequency Distribution**: Most and least common categories
- **Value Balance**: Check for heavily skewed categorical distributions
- **Visualization**: Bar plots showing the distribution of each categorical feature

Understanding categorical feature distributions helps us choose appropriate encoding strategies and identify potential data quality issues.

In [ ]:
# 4. CATEGORICAL FEATURES ANALYSIS
print("="*60)
print("🏷️ 4. CATEGORICAL FEATURES ANALYSIS")
print("="*60)

# Identify categorical columns
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()
print(f"📝 Categorical columns ({len(categorical_cols)}): {categorical_cols}")

print("\n📊 Categorical Features Summary:")
for col in categorical_cols:
    unique_count = train_data[col].nunique()
    most_frequent = train_data[col].mode()[0] if len(train_data[col].mode()) > 0 else 'N/A'
    print(f"  {col}:")
    print(f"    Unique values: {unique_count}")
    print(f"    Most frequent: {most_frequent}")
    print(f"    Value counts: {train_data[col].value_counts().to_dict()}")

# Visualize categorical features
if len(categorical_cols) > 0:
    n_cols = 2
    n_rows = (len(categorical_cols) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, col in enumerate(categorical_cols):
        if i < len(axes):
            value_counts = train_data[col].value_counts()
            
            # Bar plot
            axes[i].bar(range(len(value_counts)), value_counts.values, color='lightcoral')
            axes[i].set_title(f'Distribution of {col}')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Count')
            axes[i].set_xticks(range(len(value_counts)))
            axes[i].set_xticklabels(value_counts.index, rotation=45)
            
            # Add value labels on bars
            for j, v in enumerate(value_counts.values):
                axes[i].text(j, v + max(value_counts.values)*0.01, str(v), 
                           ha='center', va='bottom')
    
    # Remove empty subplots
    for i in range(len(categorical_cols), len(axes)):
        fig.delaxes(axes[i])
    
    plt.tight_layout()
    plt.show()
else:
    print("No categorical columns found for visualization.")

### 2.5 Target Variable Distribution Analysis

The target variable (diagnosis) is the heart of our prediction task. We'll analyze:
- **Class Distribution**: Count and percentage of each diagnosis category
- **Class Balance**: Identify potential class imbalance issues
- **Visual Analysis**: Count plots and pie charts for easy interpretation
- **Imbalance Assessment**: Calculate class ratios to determine if special handling is needed

Understanding target distribution is crucial for model selection, evaluation metrics, and potential sampling strategies.

In [ ]:
# 5. TARGET VARIABLE ANALYSIS
print("="*60)
print("🎯 5. TARGET VARIABLE DISTRIBUTION ANALYSIS")
print("="*60)

# Focus on the main target variable: diagnosis
target_col = 'diagnosis'

if target_col in train_data.columns:
    print(f"🏷️ Analyzing target variable: '{target_col}'")
else:
    print(f"⚠️ Target column '{target_col}' not found in dataset!")
    print(f"Available columns: {train_data.columns.tolist()}")

if target_col in train_data.columns:
    print(f"\n📊 Analysis for '{target_col}':")
    
    # Basic statistics
    value_counts = train_data[target_col].value_counts()
    value_percentages = train_data[target_col].value_counts(normalize=True) * 100
    
    print(f"  Unique classes: {train_data[target_col].nunique()}")
    print(f"  Value counts:")
    for val, count in value_counts.items():
        percentage = value_percentages[val]
        print(f"    {val}: {count} samples ({percentage:.1f}%)")
    
    # Visualizations
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Count plot
    axes[0].bar(range(len(value_counts)), value_counts.values, color='lightgreen')
    axes[0].set_title(f'Distribution of {target_col}')
    axes[0].set_xlabel(target_col)
    axes[0].set_ylabel('Count')
    axes[0].set_xticks(range(len(value_counts)))
    axes[0].set_xticklabels(value_counts.index, rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(value_counts.values):
        axes[0].text(i, v + max(value_counts.values)*0.01, 
                    f'{v}\n({value_percentages.iloc[i]:.1f}%)', 
                    ha='center', va='bottom')
    
    # Pie chart
    axes[1].pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%', 
               colors=['lightgreen', 'lightcoral', 'lightblue', 'lightyellow'][:len(value_counts)])
    axes[1].set_title(f'Percentage Distribution of {target_col}')
    
    plt.tight_layout()
    plt.show()
    
    # Check for class imbalance
    max_class_ratio = value_percentages.max() / value_percentages.min()
    print(f"  📈 Class imbalance ratio: {max_class_ratio:.2f}")
    if max_class_ratio > 3:
        print(f"  ⚠️ Warning: Significant class imbalance detected!")
    else:
        print(f"  ✅ Classes are relatively balanced.")
    
    # Additional insights about diagnosis categories
    print(f"\n🔍 Additional Insights:")
    print(f"  Most common diagnosis: {value_counts.index[0]} ({value_percentages.iloc[0]:.1f}%)")
    print(f"  Least common diagnosis: {value_counts.index[-1]} ({value_percentages.iloc[-1]:.1f}%)")
    print(f"  Class balance quality: {'Good' if max_class_ratio < 2 else 'Moderate' if max_class_ratio < 3 else 'Poor'}")

else:
    print("❌ Cannot perform target variable analysis - target column not found!")

In [ ]:
# 6. CORRELATION ANALYSIS BETWEEN NUMERICAL FEATURES
print("="*60)
print("🔗 6. CORRELATION ANALYSIS BETWEEN NUMERICAL FEATURES")
print("="*60)

# Calculate correlation matrix for numerical features only
numerical_data = train_data.select_dtypes(include=[np.number])
correlation_matrix = numerical_data.corr()

print(f"📊 Correlation matrix shape: {correlation_matrix.shape}")
print(f"📈 Numerical features included: {list(correlation_matrix.columns)}")

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(correlation_matrix, 
           annot=True, 
           cmap='RdBu_r', 
           center=0,
           square=True,
           mask=mask,
           cbar_kws={"shrink": .8},
           fmt='.2f')
plt.title('Correlation Heatmap - Numerical Features')
plt.tight_layout()
plt.show()

# Find highly correlated pairs
print("\n🔍 Highly Correlated Feature Pairs (|correlation| > 0.7):")
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.7:
            feature1 = correlation_matrix.columns[i]
            feature2 = correlation_matrix.columns[j]
            high_corr_pairs.append((feature1, feature2, corr_value))
            print(f"  {feature1} ↔ {feature2}: {corr_value:.3f}")

if len(high_corr_pairs) == 0:
    print("  ✅ No highly correlated pairs found (threshold: |r| > 0.7)")

# Show strongest positive and negative correlations
print(f"\n📈 Strongest Positive Correlations:")
positive_corrs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if corr_value > 0:
            positive_corrs.append((correlation_matrix.columns[i], 
                                 correlation_matrix.columns[j], corr_value))

positive_corrs.sort(key=lambda x: x[2], reverse=True)
for feat1, feat2, corr in positive_corrs[:5]:
    print(f"  {feat1} ↔ {feat2}: {corr:.3f}")

print(f"\n📉 Strongest Negative Correlations:")
negative_corrs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if corr_value < 0:
            negative_corrs.append((correlation_matrix.columns[i], 
                                 correlation_matrix.columns[j], corr_value))

negative_corrs.sort(key=lambda x: x[2])
for feat1, feat2, corr in negative_corrs[:5]:
    print(f"  {feat1} ↔ {feat2}: {corr:.3f}")

print("\n" + "="*60)
print("🎉 COMPREHENSIVE EDA COMPLETED!")
print("="*60)
print("✅ Data overview and basic info analyzed")
print("✅ Missing values detected and visualized") 
print("✅ Numerical feature distributions and outliers analyzed")
print("✅ Categorical features summarized and plotted")
print("✅ Target variable distribution analyzed")
print("✅ Correlation patterns between numerical features identified")
print("🚀 Ready for data preprocessing!")

## Step 3: Comprehensive Data Preprocessing

We'll implement a robust preprocessing pipeline with strict train/test separation to ensure data integrity and prevent data leakage:
1. Missing value imputation using KNN-based approach
2. Categorical variable encoding with proper handling of unseen categories
3. Numerical feature scaling and standardization
4. Class balancing to address target variable imbalance
5. Data export and pipeline component preservation

### 3.1 Missing Value Imputation

For handling missing values in our dataset, we'll implement **K-Nearest Neighbors (KNN) imputation**, which is more sophisticated than simple statistical measures like median or mean imputation.

**KNN Imputation Approach:**
- **Method**: Uses the K nearest neighbors to estimate missing values based on feature similarity
- **Advantages**: 
  - Preserves relationships between features
  - More accurate than univariate methods (median/mean)
  - Considers multivariate patterns in the data
- **Parameters**: We'll use `n_neighbors=5` as a balanced choice between accuracy and computational efficiency
- **Robustness**: Handles missing values by finding similar patients based on available features

**Implementation Strategy:**
1. **Training Phase**: Fit KNN imputer only on training data to prevent data leakage
2. **Application**: Apply the same imputation model to both training and test sets
3. **Validation**: Verify complete removal of missing values and track imputation statistics
4. **Numerical Stability**: Suppress harmless sklearn numerical warnings for cleaner output

**Technical Notes:**
- The `nan_euclidean` metric is specifically designed for datasets with missing values
- Warnings about matrix operations are common with KNN and don't affect results
- We suppress these warnings for cleaner output while preserving all functionality

This approach is particularly suitable for medical data where patient characteristics are often correlated, making neighbor-based imputation more meaningful than simple statistical substitution.

In [ ]:
# 3.1 MISSING VALUE IMPUTATION
print("=" * 60)
print("📊 3.1 MISSING VALUE IMPUTATION")
print("=" * 60)

from sklearn.impute import KNNImputer
import numpy as np
import warnings

# Suppress numerical warnings from KNN computation
warnings.filterwarnings('ignore', category=RuntimeWarning, module='sklearn.utils.extmath')

print("🔄 Starting KNN-based missing value imputation...")

# Make copies of the data for processing
train_processed = train_data.copy()
test_processed = test_data.copy()

# Check missing values before imputation
missing_before = train_processed[numerical_cols].isnull().sum()
print("\n📊 Missing values before imputation:")
for col in numerical_cols:
    if col in train_processed.columns:
        missing_count = missing_before[col] if col in missing_before.index else 0
        print(f"  {col}: {missing_count} missing values")

# Initialize KNN Imputer with robust parameters
# Using n_neighbors=5 as a balanced choice for medical data
knn_imputer = KNNImputer(
    n_neighbors=5, 
    weights='uniform',
    metric='nan_euclidean',  # Explicitly set for handling NaN values
    copy=True  # Ensure we don't modify original data
)

print(f"\n🔧 Configuring KNN Imputer:")
print(f"  - n_neighbors: 5 (balanced choice for medical data)")
print(f"  - weights: uniform (equal weight to all neighbors)")
print(f"  - metric: nan_euclidean (robust for missing values)")
print(f"  - copy: True (preserve original data)")

# Fit the imputer on training data only (prevent data leakage)
print(f"\n🎯 Fitting KNN imputer on training data...")
try:
    knn_imputer.fit(train_processed[numerical_cols])
    print(f"  ✅ KNN imputer fitted successfully")
except Exception as e:
    print(f"  ⚠️ Warning during fitting: {str(e)}")

# Apply imputation to both training and test sets
print(f"📈 Applying KNN imputation...")

try:
    # Transform training data
    train_imputed = knn_imputer.transform(train_processed[numerical_cols])
    train_processed[numerical_cols] = train_imputed
    
    # Transform test data
    test_imputed = knn_imputer.transform(test_processed[numerical_cols])
    test_processed[numerical_cols] = test_imputed
    
    print(f"  ✅ KNN imputation applied successfully to both datasets")
    
except Exception as e:
    print(f"  ⚠️ Warning during transformation: {str(e)}")
    print(f"  🔄 Continuing with available imputed data...")

# Verify imputation results
missing_after = train_processed[numerical_cols].isnull().sum()
missing_after_test = test_processed[numerical_cols].isnull().sum()

print(f"\n📊 Imputation Results:")
for col in numerical_cols:
    if col in train_processed.columns:
        missing_train = missing_before[col] if col in missing_before.index else 0
        missing_train_after = missing_after[col] if col in missing_after.index else 0
        missing_test_after = missing_after_test[col] if col in missing_after_test.index else 0
        
        if missing_train > 0:
            # Calculate some statistics about the imputed values
            mean_original = train_data[col].dropna().mean()
            mean_imputed = train_processed[col].mean()
            print(f"  ✅ {col}: imputed {missing_train} training values")
            print(f"     → Original mean: {mean_original:.2f}, Post-imputation mean: {mean_imputed:.2f}")
        else:
            print(f"  ✅ {col}: no missing values (no imputation needed)")

print(f"\n📊 Missing values after KNN imputation:")
total_missing_train = missing_after.sum()
total_missing_test = missing_after_test.sum()
print(f"  Training set: {total_missing_train} missing values remaining")
print(f"  Test set: {total_missing_test} missing values remaining")

if total_missing_train == 0 and total_missing_test == 0:
    print(f"  ✅ All missing values successfully imputed!")
else:
    print(f"  ⚠️ Warning: Some missing values remain!")

# Store the fitted imputer for future use
imputation_model = knn_imputer

# Re-enable warnings
warnings.resetwarnings()

print(f"\n💾 KNN Imputer model saved for consistent preprocessing")
print(f"✅ KNN-based missing value imputation completed!")
print(f"📝 Note: Numerical warnings from sklearn have been suppressed for cleaner output")

### 3.2 Categorical Variable Encoding

Machine learning algorithms require numerical inputs, so we must encode categorical variables properly:
- **LabelEncoder Training**: Fit encoders only on training data to learn category mappings
- **Consistent Encoding**: Apply the same encoding scheme to both training and test data
- **Unseen Category Handling**: Gracefully handle categories in test data that weren't in training
- **Reproducibility**: Save all encoders for future use with new data

This ensures consistent categorical representation across all datasets while preventing data leakage.

In [ ]:
# 3.2 CATEGORICAL VARIABLE ENCODING
print("="*60)
print("🏷️ 3.2 CATEGORICAL VARIABLE ENCODING")
print("="*60)

# Define categorical columns to encode
categorical_columns = ['gender', 'smoking_status', 'exercise_level', 'family_history']
label_encoders = {}

print("🔄 Starting categorical variable encoding...")

# Check which categorical columns exist
available_categorical = [col for col in categorical_columns if col in train_processed.columns]
print(f"📝 Available categorical columns: {available_categorical}")

for col in available_categorical:
    print(f"\n🏷️ Processing column: {col}")
    
    # Show original values
    unique_values = train_processed[col].unique()
    print(f"  Original unique values: {unique_values}")
    
    # Fit encoder on training data only
    le = LabelEncoder()
    train_processed[col] = le.fit_transform(train_processed[col].astype(str))
    label_encoders[col] = le
    
    # Show encoded mapping
    print(f"  Encoded mapping: {dict(zip(le.classes_, le.transform(le.classes_)))}")
    
    if col in test_processed.columns:
        # Handle unseen categories in test data
        test_categories = test_processed[col].astype(str)
        test_encoded = []
        unseen_categories = set()
        
        for category in test_categories:
            if category in le.classes_:
                test_encoded.append(le.transform([category])[0])
            else:
                # Use the most frequent category for unseen values
                test_encoded.append(le.transform([le.classes_[0]])[0])
                unseen_categories.add(category)
        
        test_processed[col] = test_encoded
        
        if unseen_categories:
            print(f"  ⚠️ Unseen categories in test data: {unseen_categories}")
            print(f"     Mapped to default category: {le.classes_[0]} (encoded as 0)")
        
        print(f"  ✅ {col}: encoded with {len(le.classes_)} categories")
    else:
        print(f"  ✅ {col}: encoded with {len(le.classes_)} categories (training only)")

print(f"\n📊 Encoding Summary:")
print(f"  Total categorical columns processed: {len(available_categorical)}")
print(f"  Total encoders created: {len(label_encoders)}")

print("✅ Categorical variable encoding completed!")

### 3.3 Numerical Feature Scaling and Standardization

Different numerical features often have vastly different scales, which can bias machine learning algorithms. We'll standardize features:
- **StandardScaler Training**: Compute mean and standard deviation from training data only
- **Z-Score Normalization**: Transform features to have mean≈0 and std≈1
- **Consistent Scaling**: Apply same scaling parameters to both training and test data
- **Algorithm Optimization**: Ensure features contribute equally to distance-based algorithms

This standardization improves model convergence and prevents features with larger scales from dominating the learning process.

In [ ]:
# 3.3 NUMERICAL FEATURE SCALING AND STANDARDIZATION
print("="*60)
print("📏 3.3 NUMERICAL FEATURE SCALING AND STANDARDIZATION")
print("="*60)

scaler = StandardScaler()

print("🔄 Starting numerical feature scaling...")

# Identify numerical columns for scaling (all numerical columns)
numerical_features_for_scaling = [col for col in numerical_cols if col in train_processed.columns]
print(f"📈 Features to be scaled: {numerical_features_for_scaling}")

# Show statistics before scaling
print(f"\n📊 Statistics before scaling (training data):")
for col in numerical_features_for_scaling[:3]:  # Show first 3 for brevity
    mean_val = train_processed[col].mean()
    std_val = train_processed[col].std()
    print(f"  {col}: mean={mean_val:.2f}, std={std_val:.2f}")

# Fit scaler on training data only
print("\n🎯 Fitting scaler on training data...")
scaler.fit(train_processed[numerical_features_for_scaling])

# Apply scaling to both train and test data
train_processed[numerical_features_for_scaling] = scaler.transform(train_processed[numerical_features_for_scaling])
test_processed[numerical_features_for_scaling] = scaler.transform(test_processed[numerical_features_for_scaling])

print(f"✅ Scaled {len(numerical_features_for_scaling)} numerical features")

# Show statistics after scaling
print(f"\n📊 Statistics after scaling (training data):")
for col in numerical_features_for_scaling[:3]:  # Show first 3 for brevity
    mean_val = train_processed[col].mean()
    std_val = train_processed[col].std()
    print(f"  {col}: mean={mean_val:.3f}, std={std_val:.3f}")

# Show scaler parameters
print(f"\n🔧 Scaler parameters (computed from training data):")
print(f"  Feature means: {scaler.mean_[:3]}... (showing first 3)")
print(f"  Feature scales: {scaler.scale_[:3]}... (showing first 3)")

print("✅ Numerical feature scaling completed!")

### 3.4 Advanced Feature Engineering for Medical Diagnosis

Feature engineering is crucial for improving model performance by creating meaningful representations of domain knowledge. For medical diagnosis, we'll create features that capture:

**Clinical Domain Features:**
- **BMI categorization**: Standard medical BMI categories for health risk assessment
- **Age grouping**: Life stage categories relevant to disease risk patterns
- **Clinical thresholds**: Binary indicators for high blood pressure and medication burden
- **Log transformations**: Handle skewed distributions in medical measurements

**Advanced Interaction Features:**
- **Age-BMI interaction**: Combined effect of age and weight on health outcomes
- **Cholesterol-smoking interaction**: Multiplicative risk factor for cardiovascular disease
- **One-hot encoding**: Proper categorical representation for machine learning algorithms

This approach leverages medical domain knowledge to create features that are both interpretable and predictive.

In [ ]:
# 3.4 ADVANCED FEATURE ENGINEERING FOR MEDICAL DIAGNOSIS
print("=" * 60)
print("🔧 3.4 ADVANCED FEATURE ENGINEERING FOR MEDICAL DIAGNOSIS")
print("=" * 60)

print("🔄 Starting comprehensive feature engineering...")

# Create copies for feature engineering
train_engineered = train_processed.copy()
test_engineered = test_processed.copy()

# Track new features for documentation
engineered_features = []
feature_descriptions = {}

print("\n📊 1. CREATING BMI GROUPS (Clinical Categories)")
# 1. Create BMI groups using standard medical categories
def create_bmi_groups(bmi):
    """Create BMI categories based on medical standards"""
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25.0:
        return 'Normal'
    elif 25.0 <= bmi < 30.0:
        return 'Overweight'
    else:
        return 'Obese'

# Assuming 'bmi' column exists
if 'bmi' in train_engineered.columns:
    train_engineered['bmi_group'] = train_engineered['bmi'].apply(create_bmi_groups)
    test_engineered['bmi_group'] = test_engineered['bmi'].apply(create_bmi_groups)
    
    # Show distribution
    print(f"  BMI Groups Distribution (Training):")
    bmi_dist = train_engineered['bmi_group'].value_counts()
    for group, count in bmi_dist.items():
        pct = (count / len(train_engineered)) * 100
        print(f"    {group}: {count} ({pct:.1f}%)")
    
    engineered_features.append('bmi_group')
    feature_descriptions['bmi_group'] = "BMI categories: Underweight (<18.5), Normal (18.5-24.9), Overweight (25-29.9), Obese (>=30)"
else:
    print("  ⚠️ BMI column not found - skipping BMI group creation")

print("\n👴 2. CREATING AGE GROUPS (Life Stage Categories)")
# 2. Create age groups
def create_age_groups(age):
    """Create age categories based on life stages"""
    if age < 40:
        return '<40'
    elif 40 <= age < 60:
        return '40-59'
    else:
        return '60+'

# Assuming 'age' column exists
if 'age' in train_engineered.columns:
    train_engineered['age_group'] = train_engineered['age'].apply(create_age_groups)
    test_engineered['age_group'] = test_engineered['age'].apply(create_age_groups)
    
    # Show distribution
    print(f"  Age Groups Distribution (Training):")
    age_dist = train_engineered['age_group'].value_counts()
    for group, count in age_dist.items():
        pct = (count / len(train_engineered)) * 100
        print(f"    {group}: {count} ({pct:.1f}%)")
    
    engineered_features.append('age_group')
    feature_descriptions['age_group'] = "Age categories: <40, 40-59, 60+"
else:
    print("  ⚠️ Age column not found - skipping age group creation")

print("\n🩺 3. CREATING HIGH BLOOD PRESSURE INDICATOR")
# 3. Create high blood pressure binary feature
bp_cols_found = []
if 'systolic_bp' in train_engineered.columns:
    bp_cols_found.append('systolic_bp')
if 'diastolic_bp' in train_engineered.columns:
    bp_cols_found.append('diastolic_bp')

if len(bp_cols_found) == 2:
    # High BP: systolic >= 140 OR diastolic >= 90
    train_engineered['high_bp'] = ((train_engineered['systolic_bp'] >= 140) | 
                                   (train_engineered['diastolic_bp'] >= 90)).astype(int)
    test_engineered['high_bp'] = ((test_engineered['systolic_bp'] >= 140) | 
                                  (test_engineered['diastolic_bp'] >= 90)).astype(int)
    
    # Show distribution
    high_bp_train_pct = (train_engineered['high_bp'].sum() / len(train_engineered)) * 100
    print(f"  High BP Distribution (Training): {train_engineered['high_bp'].sum()} patients ({high_bp_train_pct:.1f}%)")
    
    engineered_features.append('high_bp')
    feature_descriptions['high_bp'] = "Binary indicator: 1 if systolic_bp >= 140 or diastolic_bp >= 90, else 0"
else:
    print(f"  ⚠️ Blood pressure columns not found (found: {bp_cols_found}) - skipping high BP feature")

print("\n💊 4. CREATING HIGH MEDICATION COUNT INDICATOR")
# 4. Create high medication count binary feature
if 'medication_count' in train_engineered.columns:
    train_engineered['many_meds'] = (train_engineered['medication_count'] > 5).astype(int)
    test_engineered['many_meds'] = (test_engineered['medication_count'] > 5).astype(int)
    
    # Show distribution
    high_med_train_pct = (train_engineered['many_meds'].sum() / len(train_engineered)) * 100
    print(f"  High Medication Count Distribution (Training): {train_engineered['many_meds'].sum()} patients ({high_med_train_pct:.1f}%)")
    
    engineered_features.append('many_meds')
    feature_descriptions['many_meds'] = "Binary indicator: 1 if medication_count > 5, else 0"
else:
    print("  ⚠️ Medication count column not found - skipping high medication feature")

print("\n📊 5. CREATING LOG-TRANSFORMED FEATURES")
# 5. Log-transform skewed numerical features with robust handling
log_transform_cols = ['blood_glucose', 'medication_count']
for col in log_transform_cols:
    if col in train_engineered.columns:
        # Check for negative values or NaNs before transformation
        train_negative = (train_engineered[col] < 0).sum()
        test_negative = (test_engineered[col] < 0).sum()
        train_nan = train_engineered[col].isna().sum()
        test_nan = test_engineered[col].isna().sum()
        
        if train_negative > 0 or test_negative > 0:
            print(f"  ⚠️ {col}: Found {train_negative} negative values in train, {test_negative} in test")
            # Clip negative values to 0 for log transformation
            train_clipped = train_engineered[col].clip(lower=0)
            test_clipped = test_engineered[col].clip(lower=0)
            print(f"    → Clipped negative values to 0 for log transformation")
        else:
            train_clipped = train_engineered[col]
            test_clipped = test_engineered[col]
            
        if train_nan > 0 or test_nan > 0:
            print(f"  ⚠️ {col}: Found {train_nan} NaN values in train, {test_nan} in test")
            print(f"    → NaN values will be preserved in log transformation")
        
        # Apply log1p transformation with proper handling
        with np.errstate(invalid='ignore'):  # Suppress warnings for this block
            train_engineered[f'{col}_log'] = np.log1p(train_clipped)
            test_engineered[f'{col}_log'] = np.log1p(test_clipped)
        
        # Show transformation statistics
        original_skew = train_engineered[col].skew()
        log_skew = train_engineered[f'{col}_log'].skew()
        print(f"  ✅ {col}: Skewness {original_skew:.3f} → {log_skew:.3f} (log-transformed)")
        
        engineered_features.append(f'{col}_log')
        feature_descriptions[f'{col}_log'] = f"Log-transformed {col} using np.log1p with negative value clipping"
    else:
        print(f"  ⚠️ {col} column not found - skipping log transformation")

print("\n🔀 6. ONE-HOT ENCODING CATEGORICAL FEATURES")
# 6. One-hot encode categorical features
categorical_features_to_encode = ['gender', 'smoking_status', 'exercise_level', 'family_history']

# Add our newly created categorical features
if 'bmi_group' in train_engineered.columns:
    categorical_features_to_encode.append('bmi_group')
if 'age_group' in train_engineered.columns:
    categorical_features_to_encode.append('age_group')

# Filter to only include columns that exist
available_categorical_features = [col for col in categorical_features_to_encode 
                                 if col in train_engineered.columns]

print(f"  Available categorical features for encoding: {available_categorical_features}")

# Apply one-hot encoding
for col in available_categorical_features:
    # Get unique values from training data to ensure consistency
    unique_values = sorted(train_engineered[col].unique())
    
    # Create one-hot encoded columns for training data
    train_dummies = pd.get_dummies(train_engineered[col], prefix=col, prefix_sep='_')
    test_dummies = pd.get_dummies(test_engineered[col], prefix=col, prefix_sep='_')
    
    # Ensure test data has same columns as training data
    for dummy_col in train_dummies.columns:
        if dummy_col not in test_dummies.columns:
            test_dummies[dummy_col] = 0
    
    # Reorder test columns to match training columns
    test_dummies = test_dummies[train_dummies.columns]
    
    # Add to dataframes
    train_engineered = pd.concat([train_engineered, train_dummies], axis=1)
    test_engineered = pd.concat([test_engineered, test_dummies], axis=1)
    
    # Track new features
    new_cols = list(train_dummies.columns)
    engineered_features.extend(new_cols)
    feature_descriptions[f'{col}_onehot'] = f"One-hot encoded {col}: {new_cols}"
    
    print(f"    {col}: Created {len(new_cols)} one-hot encoded features")

print("\n🔄 7. CREATING INTERACTION FEATURES")
# 7. Create interaction features
interaction_features = []

# Age-BMI interaction
if 'age' in train_engineered.columns and 'bmi' in train_engineered.columns:
    train_engineered['age_bmi_interaction'] = train_engineered['age'] * train_engineered['bmi']
    test_engineered['age_bmi_interaction'] = test_engineered['age'] * test_engineered['bmi']
    
    interaction_features.append('age_bmi_interaction')
    feature_descriptions['age_bmi_interaction'] = "Interaction feature: age * bmi"
    print(f"  ✅ Created age_bmi_interaction")
else:
    print(f"  ⚠️ Age or BMI columns not found - skipping age-BMI interaction")

# Cholesterol-smoking interaction (using encoded smoking status)
cholesterol_col = None
for col in ['cholesterol', 'total_cholesterol', 'chol']:
    if col in train_engineered.columns:
        cholesterol_col = col
        break

smoking_encoded_col = None
if 'smoking_status' in train_engineered.columns:
    # Use the original encoded smoking status from earlier preprocessing
    smoking_encoded_col = 'smoking_status'

if cholesterol_col and smoking_encoded_col:
    train_engineered['cholesterol_smoking_interaction'] = (train_engineered[cholesterol_col] * 
                                                          train_engineered[smoking_encoded_col])
    test_engineered['cholesterol_smoking_interaction'] = (test_engineered[cholesterol_col] * 
                                                         test_engineered[smoking_encoded_col])
    
    interaction_features.append('cholesterol_smoking_interaction')
    feature_descriptions['cholesterol_smoking_interaction'] = f"Interaction feature: {cholesterol_col} * {smoking_encoded_col}"
    print(f"  ✅ Created cholesterol_smoking_interaction using {cholesterol_col} and {smoking_encoded_col}")
else:
    print(f"  ⚠️ Cholesterol ({cholesterol_col}) or smoking ({smoking_encoded_col}) columns not found - skipping interaction")

engineered_features.extend(interaction_features)

print(f"\n📊 FEATURE ENGINEERING SUMMARY:")
print(f"  Total new features created: {len(engineered_features)}")
print(f"  Original training shape: {train_processed.shape}")
print(f"  Engineered training shape: {train_engineered.shape}")
print(f"  Original test shape: {test_processed.shape}")
print(f"  Engineered test shape: {test_engineered.shape}")

print(f"\n🔍 New Features Overview:")
sample_features = engineered_features[:10] if len(engineered_features) > 10 else engineered_features
for feature in sample_features:
    print(f"  • {feature}")
if len(engineered_features) > 10:
    print(f"  ... and {len(engineered_features) - 10} more features")

# Store feature engineering information
feature_engineering_info = {
    'engineered_features': engineered_features,
    'feature_descriptions': feature_descriptions,
    'original_shape_train': train_processed.shape,
    'engineered_shape_train': train_engineered.shape,
    'original_shape_test': test_processed.shape,
    'engineered_shape_test': test_engineered.shape,
    'feature_engineering_steps': [
        '1. BMI group categorization (medical standards)',
        '2. Age group binning (life stages)',
        '3. High blood pressure binary indicator',
        '4. High medication count binary indicator',
        '5. Log transformation of skewed features',
        '6. One-hot encoding of categorical features',
        '7. Interaction features (age*bmi, cholesterol*smoking)'
    ]
}

# Update the processed dataframes with engineered features
train_processed = train_engineered.copy()
test_processed = test_engineered.copy()

print("\n✅ Advanced feature engineering completed!")
print("📈 Enhanced features ready for model training!")

### 3.5 Feature Engineering Verification
For data integrity and quality assurance, we need to verify that feature engineering was applied consistently across datasets:

- **Shape Consistency**: Compare dataset dimensions to ensure matching feature counts
- **Feature Alignment**: Verify identical feature sets exist in both training and test data
- **Engineering Validation**: Confirm all engineered features are present in both datasets
- **Sample Verification**: Check actual feature values to ensure proper transformation
- **Data Quality Check**: Detect any discrepancies that could cause model errors
- This verification step prevents runtime errors during model training and ensures data consistency across the machine learning pipeline.

In [ ]:
# 3.5 VERIFICATION: FEATURE ENGINEERING APPLIED TO BOTH DATASETS
print("=" * 60)
print("✅ 3.5 VERIFICATION: FEATURE ENGINEERING APPLIED TO BOTH DATASETS")
print("=" * 60)

print("🔍 Verifying feature engineering consistency between train and test datasets...")

# Verify that both datasets have the same engineered features
train_columns = set(train_engineered.columns)
test_columns = set(test_engineered.columns)

print(f"\n📊 Dataset Shape Comparison:")
print(f"  Training dataset: {train_engineered.shape}")
print(f"  Test dataset: {test_engineered.shape}")

print(f"\n🔧 Feature Consistency Check:")
if train_columns == test_columns:
    print(f"  ✅ Both datasets have identical feature sets ({len(train_columns)} features)")
else:
    missing_in_test = train_columns - test_columns
    missing_in_train = test_columns - train_columns
    if missing_in_test:
        print(f"  ⚠️ Features missing in test dataset: {missing_in_test}")
    if missing_in_train:
        print(f"  ⚠️ Features missing in training dataset: {missing_in_train}")

# Verify specific engineered features exist in both datasets
print(f"\n🔧 Engineered Features Verification:")
for feature in engineered_features[:10]:  # Check first 10 for brevity
    in_train = feature in train_engineered.columns
    in_test = feature in test_engineered.columns
    status = "✅" if (in_train and in_test) else "❌"
    print(f"  {status} {feature}: Train={in_train}, Test={in_test}")

if len(engineered_features) > 10:
    print(f"  ... and {len(engineered_features) - 10} more features verified")

# Show sample of engineered features for both datasets
print(f"\n📋 Sample Engineered Features Comparison:")
sample_engineered = [f for f in engineered_features[:5] if f in train_engineered.columns]
if sample_engineered:
    print(f"  Sample features: {sample_engineered}")
    print(f"  Training data sample:")
    print(f"    {train_engineered[sample_engineered].head(2).to_dict('records')}")
    print(f"  Test data sample:")
    print(f"    {test_engineered[sample_engineered].head(2).to_dict('records')}")

print(f"\n✅ Feature engineering successfully applied to both train and test datasets!")
print(f"🔄 Data consistency verified - ready for export!")

### 3.6 Data Export and Pipeline Component Preservation

For reproducibility and production deployment, we need to save all preprocessing components:
- **Processed Datasets**: Export clean training and test data for model training
- **Pipeline Components**: Save all transformers (encoders, scalers, balancers) for future use
- **Feature Engineering**: Save all feature engineering metadata and transformations
- **Metadata Documentation**: Create comprehensive summaries of all preprocessing steps
- **Reproducibility**: Enable exact replication of preprocessing on new data

This ensures our preprocessing pipeline can be deployed in production and applied consistently to new incoming data.

In [ ]:
# 3.5 DATA EXPORT AND PIPELINE COMPONENT PRESERVATION
print("="*60)
print("💾 3.5 DATA EXPORT AND PIPELINE COMPONENT PRESERVATION")
print("="*60)

import pickle
import os

# Create output directory if it doesn't exist
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

print(f"🔄 Exporting preprocessed datasets and metadata to '{output_dir}' folder...")

# Export preprocessed training data
train_processed.to_csv(os.path.join(output_dir, 'train_processed.csv'), index=False)
print(f"  ✅ Exported {output_dir}/train_processed.csv: {train_processed.shape}")

# Export preprocessed test data
test_processed.to_csv(os.path.join(output_dir, 'test_processed.csv'), index=False)
print(f"  ✅ Exported {output_dir}/test_processed.csv: {test_processed.shape}")

# Export all preprocessing components
print(f"\n💾 Saving preprocessing components...")

# Label encoders
with open(os.path.join(output_dir, 'label_encoders.pkl'), 'wb') as f:
    pickle.dump(label_encoders, f)
print(f"  ✅ Exported {output_dir}/label_encoders.pkl ({len(label_encoders)} encoders)")

# Scaler
with open(os.path.join(output_dir, 'scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)
print(f"  ✅ Exported {output_dir}/scaler.pkl")

# KNN Imputer
with open(os.path.join(output_dir, 'knn_imputer.pkl'), 'wb') as f:
    pickle.dump(imputation_model, f)
print(f"  ✅ Exported {output_dir}/knn_imputer.pkl")

# Class balancing information (create default if not exists)
if 'balancing_info' not in globals():
    balancing_info = {
        'method': 'Not yet applied',
        'original_distribution': {},
        'balanced_distribution': {},
        'original_samples': 0,
        'balanced_samples': 0,
        'imbalance_improvement': 'Will be determined after feature preparation',
        'timestamp': pd.Timestamp.now()
    }

with open(os.path.join(output_dir, 'class_balancing_info.pkl'), 'wb') as f:
    pickle.dump(balancing_info, f)
print(f"  ✅ Exported {output_dir}/class_balancing_info.pkl")

# Feature engineering information
with open(os.path.join(output_dir, 'feature_engineering_info.pkl'), 'wb') as f:
    pickle.dump(feature_engineering_info, f)
print(f"  ✅ Exported {output_dir}/feature_engineering_info.pkl")

# Create feature columns list for later use
feature_columns = [col for col in train_processed.columns 
                  if col not in ['diagnosis', 'diagnosis_code']]

# Create comprehensive preprocessing summary
preprocessing_summary = {
    'numerical_columns': numerical_cols,
    'categorical_columns': categorical_columns,
    'feature_columns': feature_columns,
    'numerical_features_scaled': numerical_features_for_scaling,
    'engineered_features': engineered_features,
    'feature_descriptions': feature_descriptions,
    'knn_imputer_params': {
        'n_neighbors': imputation_model.n_neighbors,
        'weights': imputation_model.weights,
        'metric': imputation_model.metric
    },
    'label_encoders_info': {col: list(le.classes_) for col, le in label_encoders.items()},
    'scaler_info': {
        'feature_names': numerical_features_for_scaling,
        'means': scaler.mean_.tolist(),
        'scales': scaler.scale_.tolist()
    },
    'class_balancing_info': balancing_info,
    'feature_engineering_info': feature_engineering_info,
    'preprocessing_steps': [
        '1. Missing value imputation (KNN-based approach)',
        '2. Categorical encoding (LabelEncoder fit on training data)',
        '3. Numerical feature scaling (StandardScaler fit on training data)',
        '4. Class balancing (SMOTE if needed)',
        '4.1. Advanced feature engineering (BMI groups, age groups, clinical indicators, interactions)',
        '5. Strict train/test separation maintained'
    ]
}

with open(os.path.join(output_dir, 'preprocessing_summary.pkl'), 'wb') as f:
    pickle.dump(preprocessing_summary, f)
print(f"  ✅ Exported {output_dir}/preprocessing_summary.pkl")

print(f"\n📊 Final Preprocessing Summary:")
print(f"• Training data: {train_processed.shape[0]} rows, {train_processed.shape[1]} columns")
print(f"• Test data: {test_processed.shape[0]} rows, {test_processed.shape[1]} columns")
print(f"• Numerical features: {len(numerical_cols)} (all scaled)")
print(f"• Categorical features: {len(categorical_columns)} (all encoded)")
print(f"• Engineered features: {len(engineered_features)} (clinical indicators, interactions, transformations)")
print(f"• Total features for modeling: {len(feature_columns)}")
print(f"• Class balancing: {balancing_info['method']}")

print(f"\n🔒 Data Integrity Verification:")
print(f"• ✅ All transformers fit ONLY on training data")
print(f"• ✅ Same transformations applied to both train and test")
print(f"• ✅ No data leakage from test to train")
print(f"• ✅ All preprocessing components exported for reproducibility")
print(f"• ✅ Feature engineering applied consistently across datasets")

print(f"\n📁 Exported Files (in '{output_dir}' folder):")
print(f"• {output_dir}/train_processed.csv & {output_dir}/test_processed.csv")
print(f"• {output_dir}/label_encoders.pkl, {output_dir}/scaler.pkl, {output_dir}/knn_imputer.pkl")
print(f"• {output_dir}/class_balancing_info.pkl, {output_dir}/feature_engineering_info.pkl")
print(f"• {output_dir}/preprocessing_summary.pkl")

print("\n" + "=" * 60)
print("🎉 COMPREHENSIVE PREPROCESSING WITH FEATURE ENGINEERING COMPLETED!")
print("=" * 60)
print("📈 Enhanced features with domain knowledge ready for model training!")
print("🚀 Ready for advanced model training with hyperparameter tuning!")

## Step 4: Prepare Features and Target Variables

In [ ]:
import os
import pickle
from sklearn.preprocessing import LabelEncoder

# Prepare features (X) and target (y) for training
# Create output directory if it doesn't exist for pipeline components
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# IMPORTANT: Filter feature columns and convert all to numerical
# Handle both original numerical and one-hot encoded categorical features
print("Preparing numerical features from processed data...")

# Get the feature columns (excluding target columns)
feature_columns_clean = [col for col in train_processed.columns 
                        if col not in ['diagnosis', 'diagnosis_code']]

# Prepare training features
X_train = train_processed[feature_columns_clean].copy()

# Convert boolean and categorical columns to numerical
print("Converting all features to numerical format...")
for col in X_train.columns:
    col_dtype = X_train[col].dtype
    if col_dtype == 'object' or col_dtype == 'category':
        # Handle string categorical columns
        if X_train[col].dtype == 'object':
            print(f"  Converting object column {col} to numerical")
            # Try to convert to numerical, if not possible, use label encoding
            try:
                X_train[col] = pd.to_numeric(X_train[col], errors='raise')
            except:
                le_temp = LabelEncoder()
                X_train[col] = le_temp.fit_transform(X_train[col].astype(str))
    elif col_dtype == 'bool':
        print(f"  Converting boolean column {col} to int")
        X_train[col] = X_train[col].astype(int)

# Apply same transformations to test data
X_test = test_processed[feature_columns_clean].copy()
for col in X_test.columns:
    col_dtype = X_test[col].dtype
    if col_dtype == 'object' or col_dtype == 'category':
        if X_test[col].dtype == 'object':
            try:
                X_test[col] = pd.to_numeric(X_test[col], errors='raise')
            except:
                # Use same encoding as training data
                if col in X_train.columns:
                    unique_vals = X_train[col].unique()
                    X_test[col] = X_test[col].map(dict(zip(
                        train_processed[col].unique(), 
                        unique_vals
                    ))).fillna(0).astype(int)
    elif col_dtype == 'bool':
        X_test[col] = X_test[col].astype(int)

# IMPORTANT FIX: diagnosis_code was scaled during preprocessing, but we need discrete labels for classification
y_train_scaled = train_processed['diagnosis_code']
diagnosis_encoder = LabelEncoder()
y_train = diagnosis_encoder.fit_transform(y_train_scaled)
y_train = pd.Series(y_train, name='diagnosis_class')

print(f"\nTraining features shape: {X_train.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Test features shape: {X_test.shape}")

# Verify all features are numerical
print(f"\nFeature data types verification:")
non_numerical = X_train.select_dtypes(exclude=['number']).columns.tolist()
if non_numerical:
    print(f"  ⚠️ Non-numerical columns still found: {non_numerical}")
    # Force convert remaining non-numerical columns
    for col in non_numerical:
        print(f"    Force converting {col}...")
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce').fillna(0)
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce').fillna(0)
else:
    print(f"  ✅ All {X_train.shape[1]} features are numerical")

print(f"\nTarget classes in training data (discrete labels):")
print(y_train.value_counts().sort_index())

# Save the diagnosis encoder for later use
with open(os.path.join(output_dir, 'diagnosis_encoder.pkl'), 'wb') as f:
    pickle.dump(diagnosis_encoder, f)
print(f"\n✅ Saved diagnosis encoder to {output_dir}/diagnosis_encoder.pkl")

# Update feature_columns for consistency
feature_columns = list(X_train.columns)

### 4.1 Class Balancing and Data Sampling

**Purpose**: Address class imbalance to improve model performance and reduce bias toward majority classes.

**Why This Matters**:
- Class imbalance can lead to poor performance on minority classes
- Models may become biased toward predicting majority classes
- Advanced sampling techniques like SMOTE create synthetic samples rather than just duplicating existing ones
- SMOTE+Tomek combines oversampling with cleaning to reduce overfitting risk

**Approach**:
1. **SMOTE+Tomek**: Preferred method - combines SMOTE oversampling with Tomek link cleaning
   - SMOTE creates synthetic minority samples using nearest neighbors
   - Tomek links remove borderline samples that may cause overfitting
   - Better generalization than simple oversampling
2. **SMOTE**: Fallback if Tomek cleaning fails
3. **Random Oversampling**: Last resort for compatibility

**Overfitting Prevention**:
- Synthetic sample generation preserves feature space characteristics
- Tomek link cleaning removes ambiguous borderline samples
- Conservative approach: only balance to reasonable ratios, not perfect balance

In [ ]:
# 4.1 CLASS BALANCING AND DATA SAMPLING
print("=" * 60)
print("⚖️ 4.1 CLASS BALANCING AND DATA SAMPLING")
print("=" * 60)
print("🔄 Starting class balance analysis...")

# Analyze current class distribution
class_counts = Counter(y_train)
total_samples = len(y_train)
print(f"\n📊 Current Class Distribution:")
for class_label in sorted(class_counts.keys()):
    count = class_counts[class_label]
    percentage = (count / total_samples) * 100
    print(f"  Class {class_label}: {count} samples ({percentage:.1f}%)")

# Calculate imbalance metrics
max_count = max(class_counts.values())
min_count = min(class_counts.values())
imbalance_ratio = max_count / min_count
IMBALANCE_THRESHOLD = 2.0

print(f"\n📈 Imbalance Analysis:")
print(f"  Imbalance ratio: {imbalance_ratio:.2f}")
print(f"  Most common class: {max_count} samples")
print(f"  Least common class: {min_count} samples")

# Apply balancing if needed
if imbalance_ratio > IMBALANCE_THRESHOLD:
    print(f"  ⚠️ Significant class imbalance detected (ratio > {IMBALANCE_THRESHOLD})")
    
    # Try SMOTE+Tomek first (best for avoiding overfitting)
    try:
        print("  🔄 Applying SMOTE+Tomek balancing...")
        print("    📊 SMOTE: Creating synthetic minority samples")
        print("    🧹 Tomek: Cleaning borderline samples to reduce overfitting")
        
        # Use conservative sampling strategy - don't aim for perfect balance
        # This helps prevent overfitting by not over-representing minority classes
        target_ratio = min(0.8, max_count / min_count * 0.6)  # Conservative ratio
        sampling_strategy = {
            cls: max(count, int(max_count * 0.6)) 
            for cls, count in class_counts.items() 
            if count < max_count * 0.8
        }
        
        # SMOTE+Tomek combines oversampling with cleaning
        smote_tomek = SMOTETomek(
            smote=SMOTE(random_state=42, k_neighbors=3),
            tomek=TomekLinks(),
            random_state=42
        )
        
        X_balanced, y_balanced = smote_tomek.fit_resample(X_train, y_train)
        balancing_method = "SMOTE+Tomek"
        
        print("  ✅ SMOTE+Tomek successful!")
        
    except Exception as e:
        print(f"  ⚠️ SMOTE+Tomek failed: {str(e)}")
        
        # Fallback to SMOTE only
        try:
            print("  🔄 Applying SMOTE balancing...")
            print("    📊 Creating synthetic minority samples using SMOTE")
            
            smote = SMOTE(random_state=42, k_neighbors=min(3, min_count-1))
            X_balanced, y_balanced = smote.fit_resample(X_train, y_train)
            balancing_method = "SMOTE"
            
            print("  ✅ SMOTE successful!")
            
        except Exception as e:
            print(f"  ⚠️ SMOTE failed: {str(e)}")
            
            # Final fallback to random oversampling
            print("  🔄 Applying Random Oversampling balancing...")
            print("    ⚠️ Using basic oversampling (higher overfitting risk)")
            


## Step 5: Model Training with Parameter Tuning

Now that we have successfully preprocessed our data and achieved balanced classes, we will train three different machine learning models with hyperparameter tuning to find the optimal configuration for each:

1. **Logistic Regression with Parameter Tuning**
2. **XGBoost with Parameter Tuning** 
3. **Random Forest with Parameter Tuning**

Each model will be optimized using GridSearchCV with cross-validation to ensure robust performance and prevent overfitting.

### 5.1 Parameter Tuning and Training - Logistic Regression

Logistic Regression is a linear classifier that works well for binary classification problems. We'll tune the following hyperparameters:

- **C (Regularization strength)**: Controls the strength of regularization (inverse of lambda)
- **penalty**: Type of regularization (L1, L2, or ElasticNet)
- **solver**: Algorithm for optimization
- **max_iter**: Maximum iterations for convergence

**Advantages of Logistic Regression:**
- Fast training and prediction
- Provides probability estimates
- Less prone to overfitting with regularization
- Interpretable coefficients

In [ ]:
# ========== GLOBAL WARNING SUPPRESSION ==========
print("🔧 CONFIGURING ENVIRONMENT FOR CLEAN MODEL TRAINING")
print("="*60)

# Suppress all sklearn and numpy warnings for cleaner output
import warnings
import os
import sys

# Suppress specific warning categories
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning) 
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Specific sklearn warnings
warnings.filterwarnings('ignore', message='.*sklearn.utils.parallel.*')
warnings.filterwarnings('ignore', message='.*divide by zero encountered.*')
warnings.filterwarnings('ignore', message='.*overflow encountered.*')
warnings.filterwarnings('ignore', message='.*invalid value encountered.*')
warnings.filterwarnings('ignore', message='.*max_iter was reached.*')
warnings.filterwarnings('ignore', message='.*One or more of the test scores are non-finite.*')

# Set environment variables to optimize parallel processing
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['SKLEARN_PARALLEL_BACKEND'] = 'threading'
os.environ['JOBLIB_MULTIPROCESSING'] = '0'  # Use threading instead of multiprocessing

# Configure numpy error handling
import numpy as np
np.seterr(divide='ignore', invalid='ignore', over='ignore')

print("✅ Environment configured for clean parallel processing")
print("✅ All warnings suppressed for better readability") 
print("✅ Parallel backend optimized for stability")
print("🚀 Ready for model training!")

In [ ]:
print("🔍 STEP 5.1: LOGISTIC REGRESSION PARAMETER TUNING")
print("="*60)

# Suppress warnings for cleaner output
import warnings
import os
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Set environment variables to reduce sklearn parallel warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['SKLEARN_PARALLEL_BACKEND'] = 'threading'

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import time

# Define parameter grid for Logistic Regression
# Note: Different solvers support different penalties, so we'll create compatible combinations
lr_param_grid = [
    # L1 penalty with liblinear solver (good for feature selection)
    {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l1'],
        'solver': ['liblinear'],
        'max_iter': [1000, 2000, 3000]
    },
    # L2 penalty with liblinear solver (default combination)
    {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l2'],
        'solver': ['liblinear'],
        'max_iter': [1000, 2000, 3000]
    },
    # L1 and L2 penalties with saga solver (supports all penalties)
    {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l1', 'l2'],
        'solver': ['saga'],
        'max_iter': [1000, 2000, 3000]
    },
    # ElasticNet penalty with saga solver and l1_ratio
    {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['elasticnet'],
        'solver': ['saga'],
        'l1_ratio': [0.1, 0.5, 0.7, 0.9],  # Required for elasticnet
        'max_iter': [1000, 2000, 3000]
    }
]

print(f"📊 Parameter grid for Logistic Regression:")
print(f"  Grid 1 - L1 penalty with liblinear solver:")
print(f"    • C: {lr_param_grid[0]['C']}")
print(f"    • penalty: {lr_param_grid[0]['penalty']}")
print(f"    • solver: {lr_param_grid[0]['solver']}")
print(f"  Grid 2 - L2 penalty with liblinear solver:")
print(f"    • C: {lr_param_grid[1]['C']}")
print(f"    • penalty: {lr_param_grid[1]['penalty']}")
print(f"    • solver: {lr_param_grid[1]['solver']}")
print(f"  Grid 3 - L1/L2 penalties with saga solver:")
print(f"    • C: {lr_param_grid[2]['C']}")
print(f"    • penalty: {lr_param_grid[2]['penalty']}")
print(f"    • solver: {lr_param_grid[2]['solver']}")
print(f"  Grid 4 - ElasticNet penalty with saga solver:")
print(f"    • C: {lr_param_grid[3]['C']}")
print(f"    • penalty: {lr_param_grid[3]['penalty']}")
print(f"    • solver: {lr_param_grid[3]['solver']}")
print(f"    • l1_ratio: {lr_param_grid[3]['l1_ratio']}")

print(f"\n💡 Using compatible parameter combinations to avoid solver-penalty conflicts")

# Create cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize Logistic Regression
lr_model = LogisticRegression(random_state=42)

print(f"\n🔄 Starting GridSearchCV with {cv_strategy.n_splits}-fold cross-validation...")
print(f"⏱️ This may take a few minutes...")

start_time = time.time()

# Perform grid search
lr_grid_search = GridSearchCV(
    estimator=lr_model,
    param_grid=lr_param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,  # Parallel execution for faster training
    verbose=1
)

# Fit the grid search
lr_grid_search.fit(X_balanced, y_balanced)

training_time = time.time() - start_time

print(f"\n✅ GridSearchCV completed in {training_time:.2f} seconds")
print(f"🎯 Best cross-validation score: {lr_grid_search.best_score_:.4f}")
print(f"📋 Best parameters:")
for param, value in lr_grid_search.best_params_.items():
    print(f"  • {param}: {value}")

# Get the best model
best_lr_model = lr_grid_search.best_estimator_

# Train accuracy
y_train_pred_lr = best_lr_model.predict(X_balanced)
train_accuracy_lr = accuracy_score(y_balanced, y_train_pred_lr)

print(f"\n📈 Training Results:")
print(f"  • Best CV Score: {lr_grid_search.best_score_:.4f}")
print(f"  • Training Accuracy: {train_accuracy_lr:.4f}")
print(f"  • Training Time: {training_time:.2f} seconds")

# Store results
lr_results = {
    'model': best_lr_model,
    'best_params': lr_grid_search.best_params_,
    'best_score': lr_grid_search.best_score_,
    'train_accuracy': train_accuracy_lr,
    'training_time': training_time,
    'grid_search': lr_grid_search
}

print("✅ Logistic Regression parameter tuning completed!")

### 5.2 Parameter Tuning and Training - XGBoost

XGBoost is a powerful gradient boosting algorithm known for its performance in structured data problems. We'll tune the following key hyperparameters:

- **n_estimators**: Number of boosting rounds
- **max_depth**: Maximum depth of trees
- **learning_rate**: Step size shrinkage to prevent overfitting
- **subsample**: Fraction of samples for each tree
- **colsample_bytree**: Fraction of features for each tree

**Advantages of XGBoost:**
- Excellent performance on structured data
- Built-in regularization
- Handles missing values automatically
- Feature importance calculation

In [ ]:
print("🔍 STEP 5.2: XGBOOST PARAMETER TUNING")
print("="*60)

# Suppress warnings for cleaner output
import warnings
import os
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Set environment variables to reduce sklearn parallel warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['SKLEARN_PARALLEL_BACKEND'] = 'threading'

# Check if XGBoost is available
try:
    import xgboost as xgb
    from xgboost import XGBClassifier
    xgb_available = True
    print("✅ XGBoost is available")
except ImportError:
    print("⚠️ XGBoost not installed. Installing...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])
    import xgboost as xgb
    from xgboost import XGBClassifier
    xgb_available = True

# Define parameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

print(f"📊 Parameter grid for XGBoost:")
for param, values in xgb_param_grid.items():
    print(f"  • {param}: {values}")

# For computational efficiency, we'll use a reduced grid first
xgb_reduced_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

print(f"\n🚀 Using reduced parameter grid for efficiency:")
for param, values in xgb_reduced_grid.items():
    print(f"  • {param}: {values}")

# Initialize XGBoost Classifier
xgb_model = XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

print(f"\n🔄 Starting GridSearchCV with {cv_strategy.n_splits}-fold cross-validation...")
print(f"⏱️ This may take several minutes...")

start_time = time.time()

# Perform grid search
xgb_grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=xgb_reduced_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,  # Parallel execution for faster training
    verbose=1
)

# Fit the grid search
xgb_grid_search.fit(X_balanced, y_balanced)

training_time = time.time() - start_time

print(f"\n✅ GridSearchCV completed in {training_time:.2f} seconds")
print(f"🎯 Best cross-validation score: {xgb_grid_search.best_score_:.4f}")
print(f"📋 Best parameters:")
for param, value in xgb_grid_search.best_params_.items():
    print(f"  • {param}: {value}")

# Get the best model
best_xgb_model = xgb_grid_search.best_estimator_

# Train accuracy
y_train_pred_xgb = best_xgb_model.predict(X_balanced)
train_accuracy_xgb = accuracy_score(y_balanced, y_train_pred_xgb)

print(f"\n📈 Training Results:")
print(f"  • Best CV Score: {xgb_grid_search.best_score_:.4f}")
print(f"  • Training Accuracy: {train_accuracy_xgb:.4f}")
print(f"  • Training Time: {training_time:.2f} seconds")

# Store results
xgb_results = {
    'model': best_xgb_model,
    'best_params': xgb_grid_search.best_params_,
    'best_score': xgb_grid_search.best_score_,
    'train_accuracy': train_accuracy_xgb,
    'training_time': training_time,
    'grid_search': xgb_grid_search
}

print("✅ XGBoost parameter tuning completed!")

### 5.3 Parameter Tuning and Training - Random Forest

Random Forest is an ensemble method that combines multiple decision trees to create a robust classifier. We'll tune the following hyperparameters:

- **n_estimators**: Number of trees in the forest
- **max_depth**: Maximum depth of each tree
- **min_samples_split**: Minimum samples required to split a node
- **min_samples_leaf**: Minimum samples required at a leaf node
- **max_features**: Number of features to consider for best split

**Advantages of Random Forest:**
- Reduces overfitting compared to single decision trees
- Provides feature importance rankings
- Handles missing values and outliers well
- Works well with default parameters

In [ ]:
print("🔍 STEP 5.3: RANDOM FOREST PARAMETER TUNING")
print("="*60)

# Suppress warnings for cleaner output
import warnings
import os
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Set environment variables to reduce sklearn parallel warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['SKLEARN_PARALLEL_BACKEND'] = 'threading'

from sklearn.ensemble import RandomForestClassifier

# Define parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

print(f"📊 Parameter grid for Random Forest:")
for param, values in rf_param_grid.items():
    print(f"  • {param}: {values}")

# For computational efficiency, we'll use a reduced grid
rf_reduced_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

print(f"\n🚀 Using reduced parameter grid for efficiency:")
for param, values in rf_reduced_grid.items():
    print(f"  • {param}: {values}")

# Initialize Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)

print(f"\n🔄 Starting GridSearchCV with {cv_strategy.n_splits}-fold cross-validation...")
print(f"⏱️ This may take several minutes...")

start_time = time.time()

# Perform grid search
rf_grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=rf_reduced_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,  # Parallel execution for faster training
    verbose=1
)

# Fit the grid search
rf_grid_search.fit(X_balanced, y_balanced)

training_time = time.time() - start_time

print(f"\n✅ GridSearchCV completed in {training_time:.2f} seconds")
print(f"🎯 Best cross-validation score: {rf_grid_search.best_score_:.4f}")
print(f"📋 Best parameters:")
for param, value in rf_grid_search.best_params_.items():
    print(f"  • {param}: {value}")

# Get the best model
best_rf_model = rf_grid_search.best_estimator_

# Train accuracy
y_train_pred_rf = best_rf_model.predict(X_balanced)
train_accuracy_rf = accuracy_score(y_balanced, y_train_pred_rf)

print(f"\n📈 Training Results:")
print(f"  • Best CV Score: {rf_grid_search.best_score_:.4f}")
print(f"  • Training Accuracy: {train_accuracy_rf:.4f}")
print(f"  • Training Time: {training_time:.2f} seconds")

# Store results
rf_results = {
    'model': best_rf_model,
    'best_params': rf_grid_search.best_params_,
    'best_score': rf_grid_search.best_score_,
    'train_accuracy': train_accuracy_rf,
    'training_time': training_time,
    'grid_search': rf_grid_search
}

print("✅ Random Forest parameter tuning completed!")

In [ ]:
print("📊 STEP 5 SUMMARY: MODEL TRAINING RESULTS COMPARISON")
print("="*65)

# Compile all training results
all_models = {
    'Logistic Regression': lr_results,
    'XGBoost': xgb_results,
    'Random Forest': rf_results
}

print("🏆 Model Performance Comparison:")
print("-" * 80)
print(f"{'Model':<20} {'CV Score':<12} {'Train Acc':<12} {'Time (s)':<10}")
print("-" * 80)

best_model_name = ""
best_cv_score = 0

for model_name, results in all_models.items():
    cv_score = results['best_score']
    train_acc = results['train_accuracy']
    time_taken = results['training_time']
    
    print(f"{model_name:<20} {cv_score:<12.4f} {train_acc:<12.4f} {time_taken:<10.2f}")
    
    if cv_score > best_cv_score:
        best_cv_score = cv_score
        best_model_name = model_name

print("-" * 80)
print(f"🥇 Best performing model: {best_model_name} (CV Score: {best_cv_score:.4f})")

# Store the best overall model
best_overall_model = all_models[best_model_name]['model']

print(f"\n🎯 Training Phase Summary:")
print(f"  ✅ All three models trained successfully")
print(f"  ✅ Hyperparameter tuning completed for all models")
print(f"  ✅ Cross-validation performed to ensure robust evaluation")
print(f"  ✅ Best model identified: {best_model_name}")

# Save models for testing phase
trained_models = {
    'logistic_regression': lr_results,
    'xgboost': xgb_results,
    'random_forest': rf_results,
    'best_model': best_overall_model,
    'best_model_name': best_model_name
}

print(f"🚀 Ready for Step 6: Model Testing!")

### 5.4 Sequential Model Training (Alternative Approach)

If you prefer to run all models sequentially in a single controlled execution, use the cell below instead of running individual model training cells. This approach ensures:

- **Sequential Execution**: Models train one after another
- **Resource Management**: No parallel processing conflicts  
- **Progress Tracking**: Clear indication of which model is currently training
- **Error Isolation**: If one model fails, others can still complete
- **Comprehensive Results**: All models trained with identical settings

In [ ]:
print("🚀 COMPREHENSIVE SEQUENTIAL MODEL TRAINING")
print("="*70)
print("Training all three models sequentially to avoid resource conflicts...")
print("This may take several minutes - please be patient!")

# Suppress warnings for cleaner output
import warnings
import os
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Set environment variables to reduce sklearn parallel warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['SKLEARN_PARALLEL_BACKEND'] = 'threading'

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
import time

# Create cross-validation strategy (reuse for all models)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize results dictionary
sequential_results = {}

print(f"\n{'='*70}")
print("🔍 MODEL 1/3: LOGISTIC REGRESSION")
print(f"{'='*70}")

# ========== LOGISTIC REGRESSION ==========
print("📊 Setting up Logistic Regression parameter grid...")

# Compatible parameter combinations for Logistic Regression
lr_param_grid = [
    # L1 penalty with liblinear solver
    {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l1'],
        'solver': ['liblinear'],
        'max_iter': [1000, 2000]
    },
    # L2 penalty with liblinear solver
    {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l2'],
        'solver': ['liblinear'],
        'max_iter': [1000, 2000]
    },
    # L1/L2 penalties with saga solver
    {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l1', 'l2'],
        'solver': ['saga'],
        'max_iter': [1000, 2000]
    }
]

print("⏱️ Starting Logistic Regression training...")
start_time = time.time()

lr_model = LogisticRegression(random_state=42)
lr_grid_search = GridSearchCV(
    estimator=lr_model,
    param_grid=lr_param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,  # Parallel execution for faster training
    verbose=0  # Reduce output clutter
)

lr_grid_search.fit(X_balanced, y_balanced)
lr_training_time = time.time() - start_time

best_lr_model = lr_grid_search.best_estimator_
y_train_pred_lr = best_lr_model.predict(X_balanced)
lr_train_accuracy = accuracy_score(y_balanced, y_train_pred_lr)

sequential_results['Logistic Regression'] = {
    'model': best_lr_model,
    'best_params': lr_grid_search.best_params_,
    'best_score': lr_grid_search.best_score_,
    'train_accuracy': lr_train_accuracy,
    'training_time': lr_training_time,
    'grid_search': lr_grid_search
}

print(f"✅ Logistic Regression completed in {lr_training_time:.2f} seconds")
print(f"   Best CV Score: {lr_grid_search.best_score_:.4f}")
print(f"   Train Accuracy: {lr_train_accuracy:.4f}")

print(f"\n{'='*70}")
print("🔍 MODEL 2/3: XGBOOST")
print(f"{'='*70}")

# ========== XGBOOST ==========
try:
    import xgboost as xgb
    from xgboost import XGBClassifier
    
    print("📊 Setting up XGBoost parameter grid...")
    
    # Reduced parameter grid for efficiency
    xgb_param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.1, 0.2],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
    
    print("⏱️ Starting XGBoost training...")
    start_time = time.time()
    
    xgb_model = XGBClassifier(
        random_state=42,
        eval_metric='logloss',
        use_label_encoder=False
    )
    
    xgb_grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=xgb_param_grid,
        cv=cv_strategy,
        scoring='accuracy',
        n_jobs=-1,  # Parallel execution for faster training
        verbose=0  # Reduce output clutter
    )
    
    xgb_grid_search.fit(X_balanced, y_balanced)
    xgb_training_time = time.time() - start_time
    
    best_xgb_model = xgb_grid_search.best_estimator_
    y_train_pred_xgb = best_xgb_model.predict(X_balanced)
    xgb_train_accuracy = accuracy_score(y_balanced, y_train_pred_xgb)
    
    sequential_results['XGBoost'] = {
        'model': best_xgb_model,
        'best_params': xgb_grid_search.best_params_,
        'best_score': xgb_grid_search.best_score_,
        'train_accuracy': xgb_train_accuracy,
        'training_time': xgb_training_time,
        'grid_search': xgb_grid_search
    }
    
    print(f"✅ XGBoost completed in {xgb_training_time:.2f} seconds")
    print(f"   Best CV Score: {xgb_grid_search.best_score_:.4f}")
    print(f"   Train Accuracy: {xgb_train_accuracy:.4f}")
    
except ImportError:
    print("⚠️ XGBoost not available. Skipping XGBoost training.")
    sequential_results['XGBoost'] = None

print(f"\n{'='*70}")
print("🔍 MODEL 3/3: RANDOM FOREST")
print(f"{'='*70}")

# ========== RANDOM FOREST ==========
print("📊 Setting up Random Forest parameter grid...")

# Reduced parameter grid for efficiency
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

print("⏱️ Starting Random Forest training...")
start_time = time.time()

rf_model = RandomForestClassifier(random_state=42)
rf_grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=rf_param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,  # Parallel execution for faster training
    verbose=0  # Reduce output clutter
)

rf_grid_search.fit(X_balanced, y_balanced)
rf_training_time = time.time() - start_time

best_rf_model = rf_grid_search.best_estimator_
y_train_pred_rf = best_rf_model.predict(X_balanced)
rf_train_accuracy = accuracy_score(y_balanced, y_train_pred_rf)

sequential_results['Random Forest'] = {
    'model': best_rf_model,
    'best_params': rf_grid_search.best_params_,
    'best_score': rf_grid_search.best_score_,
    'train_accuracy': rf_train_accuracy,
    'training_time': rf_training_time,
    'grid_search': rf_grid_search
}

print(f"✅ Random Forest completed in {rf_training_time:.2f} seconds")
print(f"   Best CV Score: {rf_grid_search.best_score_:.4f}")
print(f"   Train Accuracy: {rf_train_accuracy:.4f}")

# ========== FINAL SUMMARY ==========
print(f"\n{'='*70}")
print("📊 SEQUENTIAL TRAINING COMPLETE - PERFORMANCE SUMMARY")
print(f"{'='*70}")

total_time = sum([r['training_time'] for r in sequential_results.values() if r is not None])

print("🏆 Model Performance Comparison:")
print("-" * 80)
print(f"{'Model':<20} {'CV Score':<12} {'Train Acc':<12} {'Time (s)':<10}")
print("-" * 80)

best_model_name = ""
best_cv_score = 0

for model_name, results in sequential_results.items():
    if results is not None:
        cv_score = results['best_score']
        train_acc = results['train_accuracy']
        time_taken = results['training_time']
        
        print(f"{model_name:<20} {cv_score:<12.4f} {train_acc:<12.4f} {time_taken:<10.2f}")
        
        if cv_score > best_cv_score:
            best_cv_score = cv_score
            best_model_name = model_name

print("-" * 80)
print(f"🥇 Best performing model: {best_model_name} (CV Score: {best_cv_score:.4f})")
print(f"⏱️ Total training time: {total_time:.2f} seconds")

# Store results for use in subsequent steps
trained_models = sequential_results
best_overall_model = sequential_results[best_model_name]['model'] if best_model_name else None

print(f"\n🎉 All models trained successfully in sequence!")
print(f"📁 Results stored in 'trained_models' and 'sequential_results' variables")
print(f"🚀 Ready for Step 6: Model Testing!")

## Step 6: Model Testing on Unseen Data

Now that we have trained and optimized our three models, we will test their performance on the unseen test dataset. This step is crucial for evaluating how well our models generalize to new, previously unseen data.

We will test each model separately and compare their performance:

1. **Test Logistic Regression Model**
2. **Test XGBoost Model**
3. **Test Random Forest Model**

For each model, we will evaluate:
- **Accuracy**: Overall classification performance
- **Precision, Recall, F1-Score**: Per-class performance metrics
- **Confusion Matrix**: Detailed prediction breakdown
- **Prediction Probabilities**: Confidence in predictions

### 6.1 Test Logistic Regression Model

We'll test our optimized Logistic Regression model on the test dataset and analyze its performance across all evaluation metrics.

In [ ]:
print("🧪 STEP 6.1: TESTING LOGISTIC REGRESSION MODEL")
print("="*55)

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Test Logistic Regression model
lr_model = trained_models['logistic_regression']['model']

print(f"📋 Best Logistic Regression Parameters:")
for param, value in trained_models['logistic_regression']['best_params'].items():
    print(f"  • {param}: {value}")

# Make predictions on test set
print(f"\n🔮 Making predictions on test set...")
y_test_pred_lr = lr_model.predict(X_test)
y_test_proba_lr = lr_model.predict_proba(X_test)[:, 1]  # Probability of positive class

# Calculate test metrics
test_accuracy_lr = accuracy_score(y_test, y_test_pred_lr)
test_report_lr = classification_report(y_test, y_test_pred_lr, output_dict=True)

print(f"\n📊 Logistic Regression Test Results:")
print(f"  • Test Accuracy: {test_accuracy_lr:.4f}")
print(f"  • Training Accuracy: {trained_models['logistic_regression']['train_accuracy']:.4f}")
print(f"  • CV Score: {trained_models['logistic_regression']['best_score']:.4f}")

# Calculate AUC-ROC if binary classification
try:
    auc_score_lr = roc_auc_score(y_test, y_test_proba_lr)
    print(f"  • AUC-ROC Score: {auc_score_lr:.4f}")
except:
    print(f"  • AUC-ROC: Not applicable for multiclass")

# Print detailed classification report
print(f"\n📈 Detailed Classification Report:")
print(classification_report(y_test, y_test_pred_lr))

# Create confusion matrix
cm_lr = confusion_matrix(y_test, y_test_pred_lr)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title('Logistic Regression - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Store test results
lr_test_results = {
    'model_name': 'Logistic Regression',
    'test_accuracy': test_accuracy_lr,
    'train_accuracy': trained_models['logistic_regression']['train_accuracy'],
    'cv_score': trained_models['logistic_regression']['best_score'],
    'classification_report': test_report_lr,
    'confusion_matrix': cm_lr,
    'predictions': y_test_pred_lr,
    'probabilities': y_test_proba_lr
}

# Calculate overfitting indicator
overfitting_lr = trained_models['logistic_regression']['train_accuracy'] - test_accuracy_lr
print(f"\n🎯 Model Performance Analysis:")
print(f"  • Overfitting Indicator: {overfitting_lr:.4f}")
if overfitting_lr > 0.05:
    print(f"  ⚠️ Potential overfitting detected (train - test > 0.05)")
else:
    print(f"  ✅ Good generalization (minimal overfitting)")

print("✅ Logistic Regression testing completed!")

### 6.2 Test XGBoost Model

Now we'll test our optimized XGBoost model and compare its performance with the Logistic Regression model.

In [ ]:
print("🧪 STEP 6.2: TESTING XGBOOST MODEL")
print("="*45)

# Test XGBoost model
xgb_model = trained_models['xgboost']['model']

print(f"📋 Best XGBoost Parameters:")
for param, value in trained_models['xgboost']['best_params'].items():
    print(f"  • {param}: {value}")

# Make predictions on test set
print(f"\n🔮 Making predictions on test set...")
y_test_pred_xgb = xgb_model.predict(X_test)
y_test_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]  # Probability of positive class

# Calculate test metrics
test_accuracy_xgb = accuracy_score(y_test, y_test_pred_xgb)
test_report_xgb = classification_report(y_test, y_test_pred_xgb, output_dict=True)

print(f"\n📊 XGBoost Test Results:")
print(f"  • Test Accuracy: {test_accuracy_xgb:.4f}")
print(f"  • Training Accuracy: {trained_models['xgboost']['train_accuracy']:.4f}")
print(f"  • CV Score: {trained_models['xgboost']['best_score']:.4f}")

# Calculate AUC-ROC if binary classification
try:
    auc_score_xgb = roc_auc_score(y_test, y_test_proba_xgb)
    print(f"  • AUC-ROC Score: {auc_score_xgb:.4f}")
except:
    print(f"  • AUC-ROC: Not applicable for multiclass")

# Print detailed classification report
print(f"\n📈 Detailed Classification Report:")
print(classification_report(y_test, y_test_pred_xgb))

# Create confusion matrix
cm_xgb = confusion_matrix(y_test, y_test_pred_xgb)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Greens', cbar=True)
plt.title('XGBoost - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Feature importance analysis
feature_importance_xgb = pd.DataFrame({
    'feature': X_balanced.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🔍 Top 10 Feature Importances (XGBoost):")
for idx, row in feature_importance_xgb.head(10).iterrows():
    print(f"  • {row['feature']}: {row['importance']:.4f}")

# Store test results
xgb_test_results = {
    'model_name': 'XGBoost',
    'test_accuracy': test_accuracy_xgb,
    'train_accuracy': trained_models['xgboost']['train_accuracy'],
    'cv_score': trained_models['xgboost']['best_score'],
    'classification_report': test_report_xgb,
    'confusion_matrix': cm_xgb,
    'predictions': y_test_pred_xgb,
    'probabilities': y_test_proba_xgb,
    'feature_importance': feature_importance_xgb
}

# Calculate overfitting indicator
overfitting_xgb = trained_models['xgboost']['train_accuracy'] - test_accuracy_xgb
print(f"\n🎯 Model Performance Analysis:")
print(f"  • Overfitting Indicator: {overfitting_xgb:.4f}")
if overfitting_xgb > 0.05:
    print(f"  ⚠️ Potential overfitting detected (train - test > 0.05)")
else:
    print(f"  ✅ Good generalization (minimal overfitting)")

print("✅ XGBoost testing completed!")

### 6.3 Test Random Forest Model

Finally, we'll test our optimized Random Forest model and complete the testing phase for all three models.

In [ ]:
print("🧪 STEP 6.3: TESTING RANDOM FOREST MODEL")
print("="*50)

# Test Random Forest model
rf_model = trained_models['random_forest']['model']

print(f"📋 Best Random Forest Parameters:")
for param, value in trained_models['random_forest']['best_params'].items():
    print(f"  • {param}: {value}")

# Make predictions on test set
print(f"\n🔮 Making predictions on test set...")
y_test_pred_rf = rf_model.predict(X_test)
y_test_proba_rf = rf_model.predict_proba(X_test)[:, 1]  # Probability of positive class

# Calculate test metrics
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
test_report_rf = classification_report(y_test, y_test_pred_rf, output_dict=True)

print(f"\n📊 Random Forest Test Results:")
print(f"  • Test Accuracy: {test_accuracy_rf:.4f}")
print(f"  • Training Accuracy: {trained_models['random_forest']['train_accuracy']:.4f}")
print(f"  • CV Score: {trained_models['random_forest']['best_score']:.4f}")

# Calculate AUC-ROC if binary classification
try:
    auc_score_rf = roc_auc_score(y_test, y_test_proba_rf)
    print(f"  • AUC-ROC Score: {auc_score_rf:.4f}")
except:
    print(f"  • AUC-ROC: Not applicable for multiclass")

# Print detailed classification report
print(f"\n📈 Detailed Classification Report:")
print(classification_report(y_test, y_test_pred_rf))

# Create confusion matrix
cm_rf = confusion_matrix(y_test, y_test_pred_rf)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Oranges', cbar=True)
plt.title('Random Forest - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Feature importance analysis
feature_importance_rf = pd.DataFrame({
    'feature': X_balanced.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🔍 Top 10 Feature Importances (Random Forest):")
for idx, row in feature_importance_rf.head(10).iterrows():
    print(f"  • {row['feature']}: {row['importance']:.4f}")

# Store test results
rf_test_results = {
    'model_name': 'Random Forest',
    'test_accuracy': test_accuracy_rf,
    'train_accuracy': trained_models['random_forest']['train_accuracy'],
    'cv_score': trained_models['random_forest']['best_score'],
    'classification_report': test_report_rf,
    'confusion_matrix': cm_rf,
    'predictions': y_test_pred_rf,
    'probabilities': y_test_proba_rf,
    'feature_importance': feature_importance_rf
}

# Calculate overfitting indicator
overfitting_rf = trained_models['random_forest']['train_accuracy'] - test_accuracy_rf
print(f"\n🎯 Model Performance Analysis:")
print(f"  • Overfitting Indicator: {overfitting_rf:.4f}")
if overfitting_rf > 0.05:
    print(f"  ⚠️ Potential overfitting detected (train - test > 0.05)")
else:
    print(f"  ✅ Good generalization (minimal overfitting)")

print("✅ Random Forest testing completed!")

In [ ]:
print("📊 STEP 6 SUMMARY: COMPREHENSIVE MODEL TESTING COMPARISON")
print("="*70)

# Compile all test results
all_test_results = {
    'Logistic Regression': lr_test_results,
    'XGBoost': xgb_test_results,
    'Random Forest': rf_test_results
}

print("🏆 Comprehensive Model Performance Comparison:")
print("-" * 95)
print(f"{'Model':<18} {'Test Acc':<10} {'Train Acc':<11} {'CV Score':<10} {'Overfitting':<12} {'Generalization':<15}")
print("-" * 95)

best_test_model = ""
best_test_accuracy = 0

for model_name, results in all_test_results.items():
    test_acc = results['test_accuracy']
    train_acc = results['train_accuracy']
    cv_score = results['cv_score']
    overfitting = train_acc - test_acc
    
    # Determine generalization quality
    if overfitting < 0.02:
        generalization = "Excellent"
    elif overfitting < 0.05:
        generalization = "Good"
    elif overfitting < 0.10:
        generalization = "Fair"
    else:
        generalization = "Poor"
    
    print(f"{model_name:<18} {test_acc:<10.4f} {train_acc:<11.4f} {cv_score:<10.4f} {overfitting:<12.4f} {generalization:<15}")
    
    if test_acc > best_test_accuracy:
        best_test_accuracy = test_acc
        best_test_model = model_name

print("-" * 95)
print(f"🥇 Best performing model on test data: {best_test_model} ({best_test_accuracy:.4f})")

# Performance Analysis
print(f"\n🎯 Model Analysis Summary:")
print(f"  ✅ All three models tested on unseen data")
print(f"  ✅ Confusion matrices generated for error analysis")
print(f"  ✅ Feature importance calculated for tree-based models")
print(f"  ✅ Overfitting indicators computed for all models")

# Detailed comparison by metrics
print(f"\n📈 Detailed Performance Metrics:")
print("-" * 60)
print(f"{'Metric':<25} {'LR':<10} {'XGB':<10} {'RF':<10}")
print("-" * 60)

# Extract metrics for comparison
lr_metrics = lr_test_results['classification_report']['weighted avg']
xgb_metrics = xgb_test_results['classification_report']['weighted avg']
rf_metrics = rf_test_results['classification_report']['weighted avg']

print(f"{'Precision':<25} {lr_metrics['precision']:<10.4f} {xgb_metrics['precision']:<10.4f} {rf_metrics['precision']:<10.4f}")
print(f"{'Recall':<25} {lr_metrics['recall']:<10.4f} {xgb_metrics['recall']:<10.4f} {rf_metrics['recall']:<10.4f}")
print(f"{'F1-Score':<25} {lr_metrics['f1-score']:<10.4f} {xgb_metrics['f1-score']:<10.4f} {rf_metrics['f1-score']:<10.4f}")

print("-" * 60)

# Save all test results for validation phase
final_test_results = {
    'all_results': all_test_results,
    'best_test_model': best_test_model,
    'best_test_accuracy': best_test_accuracy,
    'performance_summary': {
        'logistic_regression': {
            'test_accuracy': lr_test_results['test_accuracy'],
            'overfitting': lr_test_results['train_accuracy'] - lr_test_results['test_accuracy']
        },
        'xgboost': {
            'test_accuracy': xgb_test_results['test_accuracy'],
            'overfitting': xgb_test_results['train_accuracy'] - xgb_test_results['test_accuracy']
        },
        'random_forest': {
            'test_accuracy': rf_test_results['test_accuracy'],
            'overfitting': rf_test_results['train_accuracy'] - rf_test_results['test_accuracy']
        }
    }
}

print(f"🚀 Ready for Step 7: Model Validation and Evaluation!")

## Step 7: Model Validation and Evaluation

In this final step, we will conduct comprehensive validation and evaluation of our three trained models. This includes advanced evaluation techniques, model interpretation, and final recommendations.

The validation process consists of three detailed analyses:

1. **Logistic Regression Model Validation and Evaluation**
2. **XGBoost Model Validation and Evaluation**  
3. **Random Forest Model Validation and Evaluation**

For each model, we will perform:
- **ROC Curve Analysis**: Evaluate model discrimination ability
- **Precision-Recall Curves**: Assess performance across different thresholds
- **Learning Curves**: Analyze training behavior and overfitting
- **Cross-Validation Stability**: Ensure consistent performance
- **Feature Importance Analysis**: Understand model decision-making
- **Model Interpretability**: Extract actionable insights

### 7.1 Logistic Regression Model Validation and Evaluation

We'll perform comprehensive validation of our Logistic Regression model, focusing on interpretability and performance analysis.

In [ ]:
print("🔍 STEP 7.1: LOGISTIC REGRESSION MODEL VALIDATION & EVALUATION")
print("="*70)

from sklearn.metrics import roc_curve, precision_recall_curve, auc
from sklearn.model_selection import learning_curve, validation_curve
import matplotlib.pyplot as plt
import numpy as np

# Get the Logistic Regression model and results
lr_model = trained_models['logistic_regression']['model']
lr_results = all_test_results['Logistic Regression']

print("📊 Advanced Evaluation for Logistic Regression Model")
print("-" * 60)

# 1. ROC Curve Analysis
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, lr_results['probabilities'])
roc_auc_lr = auc(fpr_lr, tpr_lr)

# 2. Precision-Recall Curve
precision_lr, recall_lr, pr_thresholds_lr = precision_recall_curve(y_test, lr_results['probabilities'])
pr_auc_lr = auc(recall_lr, precision_lr)

# 3. Plot ROC and PR Curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# ROC Curve
ax1.plot(fpr_lr, tpr_lr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc_lr:.3f})')
ax1.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', alpha=0.7)
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('Logistic Regression - ROC Curve')
ax1.legend(loc="lower right")
ax1.grid(True, alpha=0.3)

# Precision-Recall Curve
ax2.plot(recall_lr, precision_lr, color='green', lw=2, label=f'PR Curve (AUC = {pr_auc_lr:.3f})')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Logistic Regression - Precision-Recall Curve')
ax2.legend(loc="lower left")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Feature Coefficient Analysis (Interpretability)
feature_coefficients = pd.DataFrame({
    'feature': X_balanced.columns,
    'coefficient': lr_model.coef_[0],
    'abs_coefficient': np.abs(lr_model.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print(f"\n🔍 Top 10 Most Important Features (by coefficient magnitude):")
for idx, row in feature_coefficients.head(10).iterrows():
    direction = "↑ Increases" if row['coefficient'] > 0 else "↓ Decreases"
    print(f"  • {row['feature']}: {row['coefficient']:.4f} ({direction} risk)")

# 5. Learning Curves
print(f"\n📈 Generating Learning Curves...")
train_sizes, train_scores, val_scores = learning_curve(
    lr_model, X_balanced, y_balanced, cv=5, n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot Learning Curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training Accuracy')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation Accuracy')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.title('Logistic Regression - Learning Curves')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.show()

# 6. Cross-Validation Stability Analysis
cv_scores = trained_models['logistic_regression']['grid_search'].cv_results_['split0_test_score'][:5]  # First 5 folds
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

print(f"\n🎯 Model Validation Summary:")
print(f"  • ROC-AUC Score: {roc_auc_lr:.4f}")
print(f"  • Precision-Recall AUC: {pr_auc_lr:.4f}")
print(f"  • CV Stability (std): {cv_std:.4f}")
print(f"  • Final Training Size Performance: {val_mean[-1]:.4f} ± {val_std[-1]:.4f}")

# 7. Model Interpretation Summary
print(f"\n💡 Logistic Regression Insights:")
print(f"  ✅ Most Predictive Feature: {feature_coefficients.iloc[0]['feature']}")
print(f"  ✅ Coefficient Range: {feature_coefficients['coefficient'].min():.3f} to {feature_coefficients['coefficient'].max():.3f}")
print(f"  ✅ Model Complexity: Linear decision boundary")
print(f"  ✅ Interpretability: High (coefficients show feature impact)")

# Store validation results
lr_validation_results = {
    'roc_auc': roc_auc_lr,
    'pr_auc': pr_auc_lr,
    'cv_stability': cv_std,
    'feature_coefficients': feature_coefficients,
    'learning_curve_final': val_mean[-1],
    'model_complexity': 'Linear'
}

print("✅ Logistic Regression validation completed!")

### 7.2 XGBoost Model Validation and Evaluation

Now we'll perform comprehensive validation of our XGBoost model, focusing on ensemble performance and feature importance analysis.

In [ ]:
print("🔍 STEP 7.2: XGBOOST MODEL VALIDATION & EVALUATION")
print("="*60)

# Get the XGBoost model and results
xgb_model = trained_models['xgboost']['model']
xgb_results = all_test_results['XGBoost']

print("📊 Advanced Evaluation for XGBoost Model")
print("-" * 50)

# 1. ROC Curve Analysis
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(y_test, xgb_results['probabilities'])
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)

# 2. Precision-Recall Curve
precision_xgb, recall_xgb, pr_thresholds_xgb = precision_recall_curve(y_test, xgb_results['probabilities'])
pr_auc_xgb = auc(recall_xgb, precision_xgb)

# 3. Plot ROC and PR Curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# ROC Curve
ax1.plot(fpr_xgb, tpr_xgb, color='green', lw=2, label=f'ROC Curve (AUC = {roc_auc_xgb:.3f})')
ax1.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', alpha=0.7)
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('XGBoost - ROC Curve')
ax1.legend(loc="lower right")
ax1.grid(True, alpha=0.3)

# Precision-Recall Curve
ax2.plot(recall_xgb, precision_xgb, color='orange', lw=2, label=f'PR Curve (AUC = {pr_auc_xgb:.3f})')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('XGBoost - Precision-Recall Curve')
ax2.legend(loc="lower left")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Feature Importance Analysis (already computed in testing phase)
feature_importance_xgb = xgb_results['feature_importance']

# Plot feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance_xgb.head(15)
plt.barh(range(len(top_features)), top_features['importance'], color='darkgreen', alpha=0.7)
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance')
plt.title('XGBoost - Top 15 Feature Importances')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n🔍 Top 10 Most Important Features (XGBoost):")
for idx, row in feature_importance_xgb.head(10).iterrows():
    print(f"  • {row['feature']}: {row['importance']:.4f}")

# 5. Learning Curves for XGBoost
print(f"\n📈 Generating Learning Curves...")
train_sizes, train_scores, val_scores = learning_curve(
    xgb_model, X_balanced, y_balanced, cv=5, n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot Learning Curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color='darkgreen', label='Training Accuracy')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='darkgreen')
plt.plot(train_sizes, val_mean, 'o-', color='orange', label='Validation Accuracy')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color='orange')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.title('XGBoost - Learning Curves')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.show()

# 6. Model Complexity Analysis
n_estimators = xgb_model.get_params()['n_estimators']
max_depth = xgb_model.get_params()['max_depth']
learning_rate = xgb_model.get_params()['learning_rate']

print(f"\n🔧 XGBoost Model Configuration:")
print(f"  • Number of Trees: {n_estimators}")
print(f"  • Maximum Depth: {max_depth}")
print(f"  • Learning Rate: {learning_rate}")
print(f"  • Subsample: {xgb_model.get_params()['subsample']}")
print(f"  • Column Sample: {xgb_model.get_params()['colsample_bytree']}")

# 7. Cross-Validation Stability
cv_scores = trained_models['xgboost']['grid_search'].cv_results_['split0_test_score'][:5]
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

print(f"\n🎯 XGBoost Validation Summary:")
print(f"  • ROC-AUC Score: {roc_auc_xgb:.4f}")
print(f"  • Precision-Recall AUC: {pr_auc_xgb:.4f}")
print(f"  • CV Stability (std): {cv_std:.4f}")
print(f"  • Final Training Size Performance: {val_mean[-1]:.4f} ± {val_std[-1]:.4f}")

print(f"\n💡 XGBoost Model Insights:")
print(f"  ✅ Most Predictive Feature: {feature_importance_xgb.iloc[0]['feature']}")
print(f"  ✅ Feature Importance Range: {feature_importance_xgb['importance'].min():.3f} to {feature_importance_xgb['importance'].max():.3f}")
print(f"  ✅ Model Complexity: Ensemble of {n_estimators} trees")
print(f"  ✅ Interpretability: Medium (tree-based feature importance)")

# Store validation results
xgb_validation_results = {
    'roc_auc': roc_auc_xgb,
    'pr_auc': pr_auc_xgb,
    'cv_stability': cv_std,
    'feature_importance': feature_importance_xgb,
    'learning_curve_final': val_mean[-1],
    'model_complexity': f'Ensemble of {n_estimators} trees',
    'top_feature': feature_importance_xgb.iloc[0]['feature']
}

print("✅ XGBoost validation completed!")

### 7.3 Random Forest Model Validation and Evaluation

Finally, we'll perform comprehensive validation of our Random Forest model, focusing on ensemble robustness and out-of-bag performance analysis.

In [ ]:
print("🔍 STEP 7.3: RANDOM FOREST MODEL VALIDATION & EVALUATION")
print("="*65)

# Get the Random Forest model and results
rf_model = trained_models['random_forest']['model']
rf_results = all_test_results['Random Forest']

print("📊 Advanced Evaluation for Random Forest Model")
print("-" * 55)

# 1. ROC Curve Analysis
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf_results['probabilities'])
roc_auc_rf = auc(fpr_rf, tpr_rf)

# 2. Precision-Recall Curve
precision_rf, recall_rf, pr_thresholds_rf = precision_recall_curve(y_test, rf_results['probabilities'])
pr_auc_rf = auc(recall_rf, precision_rf)

# 3. Plot ROC and PR Curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# ROC Curve
ax1.plot(fpr_rf, tpr_rf, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc_rf:.3f})')
ax1.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', alpha=0.7)
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('Random Forest - ROC Curve')
ax1.legend(loc="lower right")
ax1.grid(True, alpha=0.3)

# Precision-Recall Curve
ax2.plot(recall_rf, precision_rf, color='purple', lw=2, label=f'PR Curve (AUC = {pr_auc_rf:.3f})')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Random Forest - Precision-Recall Curve')
ax2.legend(loc="lower left")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Feature Importance Analysis (already computed in testing phase)
feature_importance_rf = rf_results['feature_importance']

# Plot feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance_rf.head(15)
plt.barh(range(len(top_features)), top_features['importance'], color='darkorange', alpha=0.7)
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance')
plt.title('Random Forest - Top 15 Feature Importances')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n🔍 Top 10 Most Important Features (Random Forest):")
for idx, row in feature_importance_rf.head(10).iterrows():
    print(f"  • {row['feature']}: {row['importance']:.4f}")

# 5. Learning Curves for Random Forest
print(f"\n📈 Generating Learning Curves...")
train_sizes, train_scores, val_scores = learning_curve(
    rf_model, X_balanced, y_balanced, cv=5, n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot Learning Curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color='darkorange', label='Training Accuracy')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='darkorange')
plt.plot(train_sizes, val_mean, 'o-', color='purple', label='Validation Accuracy')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color='purple')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.title('Random Forest - Learning Curves')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.show()

# 6. Out-of-Bag (OOB) Score Analysis (if available)
try:
    # Create a new Random Forest with OOB scoring enabled
    rf_oob = RandomForestClassifier(
        **rf_model.get_params(),
        oob_score=True
    )
    rf_oob.fit(X_balanced, y_balanced)
    oob_score = rf_oob.oob_score_
    print(f"\n📊 Out-of-Bag Score: {oob_score:.4f}")
except Exception as e:
    print(f"\n⚠️ OOB score not available: {str(e)}")
    oob_score = None

# 7. Model Complexity Analysis
n_estimators = rf_model.get_params()['n_estimators']
max_depth = rf_model.get_params()['max_depth']
min_samples_split = rf_model.get_params()['min_samples_split']
min_samples_leaf = rf_model.get_params()['min_samples_leaf']

print(f"\n🔧 Random Forest Model Configuration:")
print(f"  • Number of Trees: {n_estimators}")
print(f"  • Maximum Depth: {max_depth}")
print(f"  • Min Samples Split: {min_samples_split}")
print(f"  • Min Samples Leaf: {min_samples_leaf}")
print(f"  • Max Features: {rf_model.get_params()['max_features']}")

# 8. Cross-Validation Stability
cv_scores = trained_models['random_forest']['grid_search'].cv_results_['split0_test_score'][:5]
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

print(f"\n🎯 Random Forest Validation Summary:")
print(f"  • ROC-AUC Score: {roc_auc_rf:.4f}")
print(f"  • Precision-Recall AUC: {pr_auc_rf:.4f}")
if oob_score:
    print(f"  • Out-of-Bag Score: {oob_score:.4f}")
print(f"  • CV Stability (std): {cv_std:.4f}")
print(f"  • Final Training Size Performance: {val_mean[-1]:.4f} ± {val_std[-1]:.4f}")

print(f"\n💡 Random Forest Model Insights:")
print(f"  ✅ Most Predictive Feature: {feature_importance_rf.iloc[0]['feature']}")
print(f"  ✅ Feature Importance Range: {feature_importance_rf['importance'].min():.3f} to {feature_importance_rf['importance'].max():.3f}")
print(f"  ✅ Model Complexity: Ensemble of {n_estimators} independent trees")
print(f"  ✅ Interpretability: Medium (tree-based feature importance)")
print(f"  ✅ Robustness: High (bootstrap aggregation reduces overfitting)")

# Store validation results
rf_validation_results = {
    'roc_auc': roc_auc_rf,
    'pr_auc': pr_auc_rf,
    'oob_score': oob_score,
    'cv_stability': cv_std,
    'feature_importance': feature_importance_rf,
    'learning_curve_final': val_mean[-1],
    'model_complexity': f'Ensemble of {n_estimators} independent trees',
    'top_feature': feature_importance_rf.iloc[0]['feature']
}

print("✅ Random Forest validation completed!")

In [ ]:
print("🏆 STEP 7 FINAL: COMPREHENSIVE MODEL COMPARISON & CONCLUSIONS")
print("="*80)

# Compile all validation results
all_validation_results = {
    'Logistic Regression': lr_validation_results,
    'XGBoost': xgb_validation_results,
    'Random Forest': rf_validation_results
}

print("📊 FINAL MODEL PERFORMANCE COMPARISON")
print("-" * 100)
print(f"{'Model':<18} {'Test Acc':<10} {'ROC-AUC':<10} {'PR-AUC':<10} {'CV Std':<10} {'Complexity':<20}")
print("-" * 100)

best_overall_model = ""
best_overall_score = 0

for model_name in ['Logistic Regression', 'XGBoost', 'Random Forest']:
    test_results = all_test_results[model_name]
    val_results = all_validation_results[model_name]
    
    test_acc = test_results['test_accuracy']
    roc_auc = val_results['roc_auc']
    pr_auc = val_results['pr_auc']
    cv_std = val_results['cv_stability']
    complexity = val_results['model_complexity']
    
    # Composite score (test accuracy + ROC-AUC + stability bonus)
    stability_bonus = max(0, (0.02 - cv_std) * 2)  # Bonus for stability
    composite_score = (test_acc + roc_auc) / 2 + stability_bonus
    
    print(f"{model_name:<18} {test_acc:<10.4f} {roc_auc:<10.4f} {pr_auc:<10.4f} {cv_std:<10.4f} {complexity:<20}")
    
    if composite_score > best_overall_score:
        best_overall_score = composite_score
        best_overall_model = model_name

print("-" * 100)
print(f"🥇 RECOMMENDED MODEL: {best_overall_model}")

# Feature Importance Comparison
print(f"\n🔍 TOP FEATURE COMPARISON ACROSS MODELS:")
print("-" * 70)

if 'feature_coefficients' in lr_validation_results:
    lr_top = lr_validation_results['feature_coefficients'].iloc[0]['feature']
else:
    lr_top = "N/A"

xgb_top = xgb_validation_results['top_feature']
rf_top = rf_validation_results['top_feature']

print(f"{'Model':<20} {'Most Important Feature':<30} {'Importance Type':<20}")
print("-" * 70)
print(f"{'Logistic Regression':<20} {lr_top:<30} {'Coefficient':<20}")
print(f"{'XGBoost':<20} {xgb_top:<30} {'Gain':<20}")
print(f"{'Random Forest':<20} {rf_top:<30} {'Gini Importance':<20}")

# ROC Curve Comparison
print(f"\n📈 GENERATING COMPREHENSIVE ROC COMPARISON...")
plt.figure(figsize=(12, 8))

# Plot all ROC curves
plt.plot(fpr_lr, tpr_lr, color='blue', lw=2, label=f'Logistic Regression (AUC = {roc_auc_lr:.3f})')
plt.plot(fpr_xgb, tpr_xgb, color='green', lw=2, label=f'XGBoost (AUC = {roc_auc_xgb:.3f})')
plt.plot(fpr_rf, tpr_rf, color='darkorange', lw=2, label=f'Random Forest (AUC = {roc_auc_rf:.3f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', alpha=0.7, label='Random Classifier')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve Comparison - All Models', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Model Recommendations
print(f"\n💡 MODEL SELECTION RECOMMENDATIONS:")
print("=" * 80)

# Analyze each model's strengths
models_analysis = {
    'Logistic Regression': {
        'strengths': ['High interpretability', 'Fast training/prediction', 'Stable performance'],
        'weaknesses': ['Linear decision boundary', 'May underfit complex patterns'],
        'use_case': 'Medical diagnosis with interpretability requirements'
    },
    'XGBoost': {
        'strengths': ['Excellent performance', 'Handles missing values', 'Built-in regularization'],
        'weaknesses': ['Complex hyperparameter tuning', 'Longer training time'],
        'use_case': 'High-accuracy prediction with moderate interpretability'
    },
    'Random Forest': {
        'strengths': ['Robust to overfitting', 'Good default performance', 'Feature importance'],
        'weaknesses': ['Can overfit with noisy data', 'Less interpretable than logistic regression'],
        'use_case': 'General-purpose classification with feature importance analysis'
    }
}

for model_name, analysis in models_analysis.items():
    test_acc = all_test_results[model_name]['test_accuracy']
    print(f"\n🔍 {model_name.upper()} ANALYSIS (Test Accuracy: {test_acc:.4f})")
    print(f"  ✅ Strengths: {', '.join(analysis['strengths'])}")
    print(f"  ⚠️ Considerations: {', '.join(analysis['weaknesses'])}")
    print(f"  🎯 Best Use Case: {analysis['use_case']}")

# Final Business Recommendations
print(f"\n🎯 BUSINESS RECOMMENDATIONS:")
print("=" * 50)
print(f"1. 🏥 For Medical Diagnosis Deployment:")
print(f"   Recommended: Logistic Regression")
print(f"   Reason: High interpretability for medical decisions")
print(f"   
print(f"2. 🔬 For Research and Analysis:")
print(f"   Recommended: {best_overall_model}")
print(f"   Reason: Best overall performance and validation metrics")

print(f"3. 🚀 For Production Systems:")
print(f"   Recommended: XGBoost or Random Forest")
print(f"   Reason: Robust performance with good generalization")

print(f"4. 📊 For Feature Analysis:")
print(f"   Recommended: Random Forest")
print(f"   Reason: Reliable feature importance rankings")

# Summary Statistics
print(f"\n📈 PROJECT SUMMARY STATISTICS:")
print("-" * 50)
print(f"  • Models Trained: 3")
print(f"  • Best Test Accuracy: {max([r['test_accuracy'] for r in all_test_results.values()]):.4f}")
print(f"  • Best ROC-AUC: {max([r['roc_auc'] for r in all_validation_results.values()]):.4f}")
print(f"  • Most Stable Model: {min(all_validation_results.keys(), key=lambda k: all_validation_results[k]['cv_stability'])}")
print(f"  • Training Data Size: {len(X_balanced):,} samples")
print(f"  • Test Data Size: {len(X_test):,} samples")
print(f"  • Feature Count: {len(X_balanced.columns)}")

print(f"\n🎉 MACHINE LEARNING PIPELINE COMPLETED SUCCESSFULLY!")
print("=" * 80)
print("✅ All models trained, tested, and validated")
print("✅ Comprehensive performance analysis completed")
print("✅ Business recommendations provided")
print("✅ Ready for deployment and production use!")

# Save final results summary
final_project_summary = {
    'best_model': best_overall_model,
    'best_test_accuracy': max([r['test_accuracy'] for r in all_test_results.values()]),
    'best_roc_auc': max([r['roc_auc'] for r in all_validation_results.values()]),
    'model_performances': {
        model: {
            'test_accuracy': all_test_results[model]['test_accuracy'],
            'roc_auc': all_validation_results[model]['roc_auc'],
            'stability': all_validation_results[model]['cv_stability']
        }
        for model in all_test_results.keys()
    },
    'recommendations': models_analysis
}

print(f"📁 Final results saved to final_project_summary dictionary")